################################################################################
#Licensed Materials - Property of IBM
#(C) Copyright IBM Corp. 2020
#US Government Users Restricted Rights - Use, duplication disclosure restricted
#by GSA ADP Schedule Contract with IBM Corp.
################################################################################

The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms), such agreements located in the link below.
Specifically, the Source Components and Sample Materials clause included in the License Information document for
Watson Studio Auto-generated Notebook applies to the auto-generated notebooks. 
By downloading, copying, accessing, or otherwise using the materials, you agree to the License Terms.
http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1

### IBM AutoAI Auto-Generated Notebook v1.14.1

**Note:** Notebook code generated using AutoAI will execute successfully. If code is modified or reordered,   
there is no guarantee it will successfully execute. This pipeline is optimized for the original dataset.  
The pipeline may fail or produce sub-optimium results if used with different data. For different data,  
please consider returning to AutoAI Experiments to generate a new pipeline. Please read our documentation   
for more information:   
(Cloud Platform) https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html .
(Cloud Pak For Data) https://www.ibm.com/support/knowledgecenter/SSQNUZ_3.0.0/wsj/analyze-data/autoai-notebook.html .


Before modifying the pipeline or trying to re-fit the pipeline, consider:   
The notebook converts dataframes to numpy arrays before fitting the pipeline   
(a current restriction of the preprocessor pipeline). The known_values_list is passed by reference   
and populated with categorical values during fit of the preprocessing pipeline. Delete its members before re-fitting.

### Representing Pipeline_3 


### 1. Set Up
If lightgbm or xgboost installation fails, please follow:
 - [lightgbm docs](https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html)
 - [xgboost docs](https://xgboost.readthedocs.io/en/latest/build.html)

In [ ]:

try:
    import autoai_libs
except Exception as e:
    import subprocess
    out = subprocess.check_output('pip install autoai-libs'.split(' '))
    for line in out.splitlines():
        print(line)
    import autoai_libs
import sklearn
try:
    import xgboost
except:
    print('xgboost, if needed, will be installed and imported later')
try:
    import lightgbm
except:
    print('lightgbm, if needed, will be installed and imported later')
from sklearn.cluster import FeatureAgglomeration
import numpy
from numpy import inf, nan, dtype, mean
from autoai_libs.sklearn.custom_scorers import CustomScorers
import sklearn.ensemble
from autoai_libs.cognito.transforms.transform_utils import TExtras, FC
from autoai_libs.transformers.exportable import *
from autoai_libs.utils.exportable_utils import *
from sklearn.pipeline import Pipeline
known_values_list=[]

In [ ]:
# compose a decorator to assist pipeline instantiation via import of modules and installation of packages
def decorator_retries(func):
    def install_import_retry(*args, **kwargs):
        retries = 0
        successful = False
        failed_retries = 0
        while retries < 100 and failed_retries < 10 and not successful:
            retries += 1
            failed_retries += 1
            try:
                result = func(*args, **kwargs)
                successful = True
            except Exception as e:
                estr = str(e)
                if estr.startswith('name ') and estr.endswith(' is not defined'):
                    try:
                        import importlib
                        module_name = estr.split("'")[1]
                        module = importlib.import_module(module_name)
                        globals().update({module_name: module})
                        print('import successful for ' + module_name)
                        failed_retries -= 1
                    except Exception as import_failure:
                        print('import of ' + module_name + ' failed with: ' + str(import_failure))
                        import subprocess
                        if module_name == 'lightgbm':
                            try:
                                print('attempting pip install of ' + module_name)
                                process = subprocess.Popen('pip install ' + module_name, shell=True)
                                process.wait()
                            except Exception as E:
                                print(E)
                                try:
                                    import sys
                                    print('attempting conda install of ' + module_name)
                                    process = subprocess.Popen('conda install --yes --prefix {sys.prefix} -c powerai ' + module_name, shell = True)
                                    process.wait()
                                except Exception as lightgbm_installation_error:
                                    print('lightgbm installation failed!' + lightgbm_installation_error)
                        else:
                            print('attempting pip install of ' + module_name)
                            process = subprocess.Popen('pip install ' + module_name, shell=True)
                            process.wait()
                        try:
                            print('re-attempting import of ' + module_name)
                            module = importlib.import_module(module_name)
                            globals().update({module_name: module})
                            print('import successful for ' + module_name)
                            failed_retries -= 1
                        except Exception as import_or_installation_failure:
                            print('failure installing and/or importing ' + module_name + ' error was: ' + str(
                                import_or_installation_failure))
                            raise (ModuleNotFoundError('Missing package in environment for ' + module_name +
                                                       '? Try import and/or pip install manually?'))
                elif type(e) is AttributeError:
                    if 'module ' in estr and ' has no attribute ' in estr:
                        pieces = estr.split("'")
                        if len(pieces) == 5:
                            try:
                                import importlib
                                print('re-attempting import of ' + pieces[3] + ' from ' + pieces[1])
                                module = importlib.import_module('.' + pieces[3], pieces[1])
                                failed_retries -= 1
                            except:
                                print('failed attempt to import ' + pieces[3])
                                raise (e)
                        else:
                            raise (e)
                else:
                    raise (e)
        if successful:
            print('Pipeline successfully instantiated')
        else:
            raise (ModuleNotFoundError(
                'Remaining missing imports/packages in environment? Retry cell and/or try pip install manually?'))
        return result
    return install_import_retry


### 2. Compose Pipeline

In [ ]:
# metadata necessary to replicate AutoAI scores with the pipeline
_input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'Exited', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 1, 'pn': 'P3', 'cv_num_folds': 3, 'holdout_fraction': 0.15, 'optimization_metric': 'accuracy', 'random_state': 33, 'data_source': ''}

# define a function to compose the pipeline, and invoke it
@decorator_retries
def compose_pipeline():
    import numpy
    from numpy import nan, dtype, mean
    #
    # composing steps for toplevel Pipeline
    #
    _input_metadata = {'separator': ',', 'excel_sheet': 0, 'target_label_name': 'Exited', 'learning_type': 'classification', 'subsampling': None, 'pos_label': 1, 'pn': 'P3', 'cv_num_folds': 3, 'holdout_fraction': 0.15, 'optimization_metric': 'accuracy', 'random_state': 33, 'data_source': ''}
    steps = []
    steps.append(('column_selector', autoai_libs.transformers.exportable.ColumnSelector(activate_flag=True, columns_indices_list=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])))
    #
    # composing steps for preprocessor Pipeline
    #
    preprocessor__input_metadata = None
    preprocessor_steps = []
    #
    # composing steps for preprocessor_features FeatureUnion
    #
    preprocessor_features_transformer_list = []
    #
    # composing steps for preprocessor_features_categorical Pipeline
    #
    preprocessor_features_categorical__input_metadata = None
    preprocessor_features_categorical_steps = []
    preprocessor_features_categorical_steps.append(('cat_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[1, 3, 4, 5, 6, 8, 9, 10])))
    preprocessor_features_categorical_steps.append(('cat_compress_strings', autoai_libs.transformers.exportable.CompressStrings(activate_flag=True, compress_type='hash', dtypes_list=['char_str', 'char_str', 'char_str', 'int_num', 'int_num', 'int_num', 'int_num', 'int_num'], missing_values_reference_list=['', '-', '?', nan], misslist_list=[[], [], [], [], [], [], [], []])))
    preprocessor_features_categorical_steps.append(('cat_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_categorical_steps.append(('cat_unknown_replacer', autoai_libs.transformers.exportable.NumpyReplaceUnknownValues(filling_values=nan, filling_values_list=[nan, nan, nan, nan, nan, nan, nan, nan], known_values_list=[[149007597720685508753506128426238700846, 310090853095096028219177893019669258142, 38195942481901026394256959963367713524, 83672662407872831558691699368409195536, 128269921190646408360091979559258162367, 208662180817739027228211965656007478926, 184184990703327347595972752247257261951, 305955084972885027847332827075718518294, 200663397342099851301902471945872787195, 317666274555694458651320977911476754215, 120712945804619934289095527688543305359, 209098476983538584154797488794288951844, 94126054416683936371400846900325579281, 174275379255997101889574962893136125039, 90467901797652696292054460038345614350, 218029202558309777211930406580770855644, 84768872290530747482676205586073157665, 266417070792637185715521752604386766127, 312095775317564960847339302566851732098, 168969100445071870430762055943196014338, 51841603811244679373279661769431058810, 289624265291206431189359211048771327713, 150178007070165539112943174517498120167, 26295847857222828861258286708903087185, 176320028421948979632730011331869711056, 194988411436439583730408981157272541190, 251765296420318328340900747655967737113, 318398549753152737694038318411958872905, 132237652102037471992321037788926109235, 272988029478663959720314285742645078526, 116824076335816041637972166798480391453, 131854731895291048705714036754347361748, 97316347908295857447388487549449742829, 46312430590160095087158345028820725837, 322551743527632296454572664759196335668, 229713640717605883254840512028596265357, 173414451248527866876873070749514430877, 178482040415014213646481122800886525418, 329179753615135640047462326061582008137, 116063417490861757862538566120913395218, 287708621527623841677805484413713801653, 244116435737297339137400863773914195573, 34157762602355565868139518408432579521, 312689220616328981425028354326627841647, 134334101883525746742423672901918258822, 301366535500347770147150968188056466055, 92493801909505237662527877558826669152, 113755118980222291893509203086710965845, 26715375765577483769662070497302377617, 162972054849438579494545668752024418629, 5824287872191526752698798762331297850, 215252424470712270321813716663169819122, 235269162756838290492325319585638652442, 199318686781328189304216068432625239390, 207425136152508975487988590934749007573, 137791251251191983008338600969742464005, 209434127426037366352877299581272146151, 173019435382667466579600520824432691097, 192124653850492901126530379776976891255, 31418796571437348652192366842636365449, 48747429804438879594755325935207142474, 115650925469763888536542038278286624883, 30337529502790826004362445747716994919, 51076154488856862960783522507801942891, 111914933301022520369908508457734046023, 211611392634311239461723538555965971599, 255281225955319629157272880685628815335, 178617486949103775239584092433385804595, 72794799947741402096636276847637854027, 96834796749371743038764197369176258965, 238155637949322391346026613088163194910, 65775644000421271337885928652027107275, 306964965947202094896735359816842816920, 303946498711537231771122961839265492377, 110439518576264975436511218160767394413, 275800366090995525365666683784852387814, 98824022357732017848789492766009562233, 203768510054730076284198739861999018595, 267776254144528283160642149377140310789, 289655729353230147526604423996965655953, 249184078189304060247369443367125345953, 159951160458084447796710082828349907419, 173979385429240017877020823180624142608, 159114344125430547225956463852026670470, 43790183086162912827865188591742796219, 233449610186979302528028335590644029235, 113330308822401616018594860270876080569, 132097788911347537941478373111586023714, 92547813696959726642363208447106987042, 191009384177973407689022707595143840737, 30845446638675147334825932633756803524, 220668713618815169119627727765985799912, 279538170493699471974246663973565579850, 80215261859218248374138140162909659321, 230218032615539901048220067583532143677, 171831045808025012165032212704924660492, 69159577537348021726417666203864576118, 221825086959878501409083314080821262504, 75434786864255916463259274820716274217, 265782363681202123915854686926678497861, 146852955677026390616362642956298479443, 123124100995478013092715693105333565749, 253535799205256863218103148277163650805, 265692775807836068055932994420948513037, 139985980515644598813635745718897233658, 148653736673781301483612458456997873692, 144426480844942213041630733564564417550, 211441400255206093973036422361488897401, 72583543102318038494185842004174463950, 84280636539341342869853721924062477392, 299500025421267905771733458015866920187, 112668427687248638148536085350261207815, 53393852412975517421938649004434823962, 112826030109957979317010667261963462660, 279495187726831781779666967685785438113, 186840275413334159816154507910722488494, 24461295811302466440833013632372097900, 225362151760408256432883839326446247155, 154681114302009896589731318506021137487, 3223749394212673716273691823869545660, 166703178944120445249004475008058724774, 73061926737062504217529643377146227491, 25271387802862878905517546509478537763, 48993097735246731944472024839664542022, 259402839146592298391742171619753232957, 314826791993844052983199518756819364529, 98830626443065202287502503319571055252, 218651127549062141771399754989554121267, 237292520191470664982767629397612960705, 140033887117518735812527647742553295824, 323062097113239286129477854483347087109, 66085326277732631290582446751841912375, 127439136318050842456845436558544653975, 31773829083245040460144299304343688113, 40084528067682007628681095579362007392, 70770765468408246068309676054605460993, 33203382889097699741713579572975102656, 44497520056823451541750949301526452502, 58508779346268887231148576300349573914, 301230897495340066611308496477895248264, 138227176725033647972239005313354436041, 12169135873507983043930112048835650531, 6743986147991660728915431101015228236, 301316248592225009899453558097716157432, 204310282672326748246343789718949597574, 105463168710732483720196385036205653307, 45475601117851741206111672464233379925, 97952578822040375193505115831956596083, 179525218924340736702469479127542715557, 193117382243150493746188776593890388547, 309763155920847392496211641156881273966, 323170019269767133505117546452556439488, 91134012300951195691106841837428973749, 251666495331244993526866566656655423931, 217329724562256805314127570046107360061, 305684377434856365714204093992738778506, 62021126106164346891296865389477821521, 279675755680515199334604702298703380255, 207950118445368090496242143510247281063, 112955790915600057105356656176920971160, 72308478340489330709415998667922418683, 4933489769392557685704207080159269043, 91524410434394461029314652118231078854, 185059324502585174632371461614957380605, 47798562342946760477868183996397818981, 164888838775570587547154085369049541701, 90149688944743388990678639061854672507, 257526494836081285376003235088975170010, 150666166870016536830822183459643988913, 249798872042226501059096296714413523025, 319391430903453988533329385099851667743, 33826272482944366632718675688649069367, 173766614214452999671532494994767852430, 183556016312540442130845944105095072933, 23680512750797952642147342515658946571, 40538589852981721538160220289294212841, 164759989427572885343813610173423839916, 293004002194301361986085609100141078188, 248525040431536864911828551914554414989, 339280687937210757809077405686452266378, 226981798526038165372237239137321918275, 194063532395813816208944460270646558400, 94352095251948331321682371031568784513, 86250756834369583069994110737309150062, 303351855143253427151428010436328715230, 113578994606145086470631428406377956591, 53580821070455869578308190480658404163, 292354069568292620329661649561248391293, 140084366301246682994479305997491579154, 78503163211050546251458809294939673033, 32328288125717775585146506196266872864, 193126568005358293384699157978621648819, 204171637979606783781520011371984563030, 52728146536072210165527208558853838540, 131678818463714588160183692309921203720, 135280658983903993056939217896423734564, 208112516573395051379031266057911792040, 186966556299579316204495594458889368447, 187537015903426777442074411449688905108, 124362699467537447826737927383288835553, 191374174659360192858833985472436807595, 178253909918494931441057207614327114627, 162305276328666549850274507867018965436, 223926297379833104424491674553586035992, 200232044207771390342734331358583604875, 96169129392679369406971569845025166362, 2276643886506881943177802119088498107, 255071801162517751758878844280376150152, 27223125411017391828152177729351463449, 172731516351205115997792934093961920898, 100587997298672197592770005540042081728, 8360148742654473880904563187833260975, 289543332256169184532077224922414249339, 81653215381069404596720196812255320163, 191728120721344526394198048064060987832, 39979488650703657985793484348385140943, 123301935057603743057652994722627642369, 121333713013883478449621383858378895466, 253965560166248264257801868668048675679, 304791160275093358487245504088865903187, 289913329136359009960034298116693697397, 226924265143383926638901645238716230772, 119385894173665707063201906986766375295, 160506152644952970601095724759436620965, 145376162711575020696555111136606347462, 313552923293090465480318002992710140369, 334602334924116965550550747298899902668, 34083608284258905229764767393919717733, 186999467905348603188124278690108183936, 29488153522495987759128939171023880184, 227866789153870289014938274761414918813, 171007203023136126630432012672499431676, 77948551498316883599899628642770468925, 134104208717333345385409174208351488853, 309782724290245396668082628620805636318, 64829480754962064805960671716065723689, 172097998144505038541739803541360869041, 89475326116874491189757761767758686615, 130694470483143902727646965524912878307, 257366142088187416383204424576746036974, 330293699500045766404754785867211459972, 72705613881279998886037314723597270876, 101662818930134090034414375167046480453, 96224529404243736277579551972987445789, 139006176189088250798244218896581399274, 86742414529759051421686993943665386786, 12558431783478815470819405738476062799, 127890503734348085290732044280776229836, 189645487461297082884981739581899090516, 80498073487999058279339554310444600348, 280835759942406375582494659380597829079, 171930995868480805006613124214429263658, 302998899618806832007359524062258184080, 295962332738369130475534876538266346954, 71755565455406503209846719261303139839, 22992470388803156202085732645010657862, 144754928932333989672051362008517247725, 162474292491099128372796252231594161715, 43222612875059775157273919438097633711, 63341329634536779525050640606738416744, 77742812652201784769926782893618882330, 57306813966733200524945020378781580407, 211662112539394817976393150261812074857, 1438937915182442439625455242659799829, 42777087920039375084012549624032206781, 27766273900332474942857847059949873751, 88529832488109553353137868690060478271, 328603261865463711459221514310655220131, 107840883409724169690502276841701653310, 156302301188162046387927299416813659594, 70090029663694697914527800766391881542, 97630425486651777882979330280046614933, 46753608813551609758902883658386852126, 111118036347472951394351703265198001624, 319918244900922025272684401364302545067, 221173990825359371302463844329481236274, 334411250935025641737191045781882058916, 136329700024062953756075085636752965043, 132910314523634709600888011297070080271, 232631851465603251725423368025803671570, 2336122412490514468843776203257358991, 109452976098658735155949975056017331410, 103389972075086650963114926671757941393, 275907223525381412607047472794913303962, 73488017111922366457010985066972352076, 151558290752948593647982982542615099118, 78357178094545073204317074039791389553, 41426408672147440786992693618925855321, 109430664295990245423194561762499423579, 27623926206549991426910282773071144152, 80989930114731555163037926088744548726, 166466659269241816799421791476265705932, 26822346028072780448064829675104324720, 298425774509189192861722585951782336107, 237331889945760015118143247627540001665, 76641750282894195501216826099668801384, 158708847348017236571180841302346716964, 100134849418877322587321717934048896502, 256821238181268481261709066938753603508, 105769241596291121852124483124458137114, 156803578735759583755266626953769927090, 315546195070089242525468291651028910564, 110289948400808063383994509098761875733, 106029190725423144256070578624816068876, 230847064070364862367686028843513365923, 100902206058835464020243034679672247465, 124111290822875034314212928160270689119, 78177549948099561955423102878491998084, 206256042257854220804602103475615516206, 215955379362795810249329877690103283681, 287017786768063360058288177338804797086, 44385059014744864106198263848499189182, 271381397148169865981104551531212156242, 119424234040978623787479343240027505225, 139724928138035348902776222640050522843, 223133606057681441744794133083635599341, 3518137000902128764411292597508121864, 52467389336352035513616682758124929852, 287456836248395249485021501950352186194, 314196599857493922186092762232001710890, 142686174586192034376428934551252931254, 58260929686924616800521095525037811834, 134400444117845117653962547475361777408, 308537642641884373944601328774892917295, 152304177276855386269724122713291535406, 195075020319390210196216036239163892851, 136414334289034580578315724810849004842, 20196851619994389770119832820037457756, 256400399810833536004894589963648113313, 322545413608484461304556760447899395078, 9497822652337888067159567373735215200, 27654567069009213840713502583890309071, 44657821145609651891222348418928973150, 186406068824938302442959932643983020591, 20073139090221745653225332270504265496, 23112687468966995663535124303533281320, 118381177964581501460496563465973438973, 141640877593165233268719069553264454964, 206231388694559356281832797041660172175, 319968334742405818339714366835870867243, 254807130868157839058512615613515835515, 103364994946429550238863360784601108308, 41892379267508091917580201906489144369, 174835950917784997131884128903716922021, 203045782296628621587302761744434161770, 7540598836124319732340624105987892786, 188077256496721617979152105293002580713, 45201353860100348314751518448803578413, 130177057699914698182162428362768225713, 80588570684019489672129609960243336347, 256493480636382010334667707997090267674, 239385179881658286366597618347408194123, 334739749266762722703234900814553026088, 335154183097007165382573125919514588907, 167839075327924234053487964096939337658, 28028321410049622799095235767455223176, 8610391526124060151879589121321293331, 286983756548819385860133575875232790416, 5881404781111310988841766189455851484, 173398712898760749572265768051823420434, 321818601614479562702263277298716738154, 66570994403613765639996561897552285630, 324533643401345726261139027174933693950, 168646261225496076613696084366183796526, 240702892912698703945895463396009705653, 73749839735117958488753322348353184125, 163855112141469097782711058044515698262, 226134524513400606760593265128494068508, 311799079546574342169421702988713956268, 268563463125124821470572653098593814377, 141535536759163548157199816671865787321, 73678445875701017270270774407901637906, 786957211549632056523133958403351412, 335496475694268660972172527757216533353, 66668388651190793013204144226922569915, 292631363185296462310326968166655415384, 48470309185065340014224948235121465092, 324884436150323339696430195146452373962, 139083839364834765986674494005476755456, 303221806711906283508212339374402720677, 112732647620004279186484125675903761756, 126662407120129664698462062512589647868, 247219056970914557290519918513256868435, 281390776391367759414086473633393552235, 125650183362100580891076861762109767176, 142111904711651573414038465526072455455, 274054046317718376190377046384251297923, 36311285151122301463545696675297024557, 161808219230110963577070990012209965851, 145826766885924871808519055953411630380, 331205749624890826663688881852223838538, 135020941937714169126673673486472848078, 240182213314903629696059020657647349018, 43811833731729805085817935398973326557, 181129934191530054093595290565904340194, 197326952662949665622379906413936873716, 81027378632734460801976130401792470084, 308357491467336531069227741564469070105, 161006798775375698538317146417819149201, 67310187611970939041569572888291542215, 142652489660330376468845535503970182191, 173571207464790233902966975300567441397, 39788158523220687240662553070914209215, 228212400791673243202367959853804270974, 266606198542146983218525503141352506016, 116725161369641749673177819472281873044, 79881359260331177172136150900170023268, 144699621110266828790687608497776327660, 255811770348450801573637914724396944561, 38258802358453559619126796867121498252, 259201022307579774009882664879369721351, 321085266220940911844183068572014591967, 85517747406938882087873172095977577732, 228164699520163996480827748885325149159, 312365520051126724366019763980676841027, 130142375178371224499576954282310120729, 259418968862348393966233383454045141608, 318246015427571375279931310938896472236, 298470570005963764673656651308170077578, 146750819054387616079304055425116912761, 137565856587076280946455481889825853039, 43142712541936341069042877665747430242, 9365249086048605170462573749066218698, 66034097919814017595590879450679961123, 68502301931425944758219755958221911408, 81452093535588175867211324145395113258, 46298623396655838828178668005573247650, 143582013008830734104111667080772553823, 326596547587071339543054706481376391974, 214023622932978040096936357179300528301, 211603388949410276003937374592007185264, 208875628837011528315831466640931583829, 91779330269461717484127894641501584842, 2882771684263288166134296889731491036, 309688893597499293658151988995898205805, 321261696963338087783312552417442847533, 137175742590800567111774719802641546655, 208880043245939622070976709289739938678, 25839806410664890514579245342253796299, 71816859057885421942740064830678232109, 273421909771065051666875176080956028776, 100266889276706557423138822090187813757, 297292816507152042250209316695519311038, 125622587437330249496550217153417974733, 217179481531683952243619352949474563273, 259051787439667792376950649037314349760, 310176504191117608393112244625199724406, 127868063930181732334907032210677220759, 231547705444293457353881735864449843698, 206543945298667639384228734120095989201, 212981634536421056011872440282102909304, 59020338970113876955597074316310815031, 262589413703280163213224002511739585655, 301727990756421644967066539493353297815, 161527375965485100076686387560841530155, 228750968091834796960381301213130952235, 91849844577842357621045922685248614859, 275957573772037224808212891113011002445, 48147858134941497718847751161005957779, 83373008204076570740170222906070347645, 19672055698652346972360933522536017256, 220304255437177642863391997388096529493, 148424995376181598498383675363646708684, 274339155040211674804562295695394888964, 45138164870691941247134994003757846478, 10964669545886247979566382231711215334, 65125585432471179264604615044095161734, 330363659254050542105093016992351041739, 268781950555710361291362189093465584023, 15999873561640217678219802025025188139, 294111527382819162755534493908870681419, 64411768518854761532277541885523994048, 54738793499751970514759842262816626361, 49024575900572648265690928266763084126, 309795766153163549926024732407857299381, 66083931967318797689956912747929893962, 231823436590040111884080827564767714634, 69213537240517587702978703062236610480, 81017557220366599926780496681707580396, 276151628489013251667026835942850370645, 4946439706533650409225526527097628877, 237831822487286348995881011571532361107, 298639396697958020581154912280412940920, 31460636555301125251680059809790321162, 8522376630761350158253500929299533704, 9471352987009354197710249456041913227, 41878116874520627892657521359052349449, 72399050822812931800317050093111164814, 86690136881271941398745099566962955537, 204604651872440096581799263678586731777, 25471339822014589800324110878250834051, 291599873054500785100202252726448570527, 323698379261689569724294430695035621146, 172415875662809864711712343181878134320, 278356398654230811182631467186674489510, 334903631570590923285791671202927454622, 148336157962308042526624784682675924334, 70014189999821263053233201739454747194, 237356838448369172230135153435186830488, 324243510890210423139465963630194439096, 265700168108624464923734237035529661110, 108861800121974088966561977836539068631, 97541900262099016155147899159367789454, 24998423193395901728924511941892256505, 311965670142801506029310448811975699223, 224273926195293879481850484201186106719, 123411460532991111486556182476817987841, 245104689910229031964393508142257817100, 88633350399943503629407384318392699621, 133602695464873381340418090221526855429, 58840792471081216824059926882180255953, 69025448296548697356665471304179890140, 295264807859498661403177599710881465618, 90787532802714564469723677978342115776, 252373919940838190972121975258163741163, 333122376652245560292825971725935178000, 33532833848162164829642911123070901013, 11310495927234158559902501653168258956, 37353888864225493311363952914966100255, 304049430712543203131682277751252946139, 225303774884926326952801717865111678464, 165897386495783149906894562200835580662, 338717265593338405415185903807740068126, 218571851273734790901067167104251615138, 100923943229396416031999273165805009897, 156840665595979029379894759586346299017, 23557884941799889777975668843152685495, 254695041593390709855071181703729733061, 273267242873890831372129840229659908830, 18655154338059286065980234389335750187, 86822153226344222573251111219081203037, 298075055393528348709830986880182382914, 190335244155822423583227491846887499044, 295280835842344666399648872796386380488, 221221564706630031079594431964789132489, 79804158907860816670176857872944973754, 215775579471052691171845374939905959332, 183677781523782993731855681370364203626, 96415238226014204540581725794815576032, 256388971333296444654418932786464561423, 65107352702266361813324908743527130024, 50814879900843813845360695652796701135, 141600652642458927930309043506929454376, 190677735298642495717618089056774025485, 140937300158088872476804934833855458323, 102483256985750337880839800782251485183, 90158934914841667675866309611372015588, 269580567597490574837771013963428816748, 164561936920781200759433988199167586744, 80830854783635616565252761536127081859, 307813294102930813264778683335697378210, 89931227239435328302257262490504733558, 235089199000553089409558590588701414778, 272723035210958895611601369802327064836, 264939369078399214013676474772102421063, 259323360063875409453218312103869215279, 247032798027020035222320322938652921767, 110495315757714121976036080416981410402, 68217875598717792815740758541616175546, 290095908834598528848346394550510506914, 304848438529696834523329627813700509300, 37171867577115641720172901678411675162, 63632728738897852980918143778142144297, 194612469761473302267709701546490719711, 259502773411032505286193747488223169721, 28421871726659378796605265958413267794, 41132470199039477592368890528290192449, 310164450887064093112309655498214712722, 263027324391619084739486249332381665735, 74918373300555914610846448164942561648, 290321910049915171981455265290042762114, 44044220788553166542493577772274515434, 233487053673318713901353701123050438063, 191835066340235490669367441929227616414, 97337933565275870644117254861424927197, 147963439139655372068390972412160864572, 176756217496884466098620734694072491869, 130780404980382047866273204102917903763, 265294566578797604979370088326960377065, 121870963893592660741440868071810554128, 209324430491742894117014394612823294614, 148936397425072678282122304989068684032, 149580202899081993790288999426711650179, 239783062917188077014431178407969711097, 159283965091883004952265755858171047338, 301601644306902151795487710358850312090, 141920045209258191098393083704052242342, 220151144176885074886444800080707083838, 209355205242601756671056269906295611466, 163143869822836616448040458961508167945, 176016077366839730566410666268270776768, 135638392682441084270064922663351384530, 307608634323847326876705383014102848105, 180575949859401054533402594519592540213, 110964129675444353924112145260419288241, 214271117589907890350149691096886747797, 36464474671889800644004048076329330012, 322646448797228012291959056260629882031, 100557143887285564295819335153541590525, 277125319814914355243199185204806789748, 238785328691084984884611154842684457568, 204931887618932609734331613569742459723, 300604630346615463628264604204397005645, 262730086362074084087677552351633090727, 66646125100841838744684118667826409845, 140600138867353878894987571675580265696, 113063401625802254350956751989945583273, 149195046865158073255287596723089627768, 10444274091377558350235920617681208321, 25548861703717514119958944238265678627, 83481204868473851611571066640807448379, 226687930519503596773241968203564011997, 76930883021644314709353551549546372976, 263459882630844699279406267566303311276, 338245918169757098088278922535557090160, 59829176772760864396352089935010459625, 151519920133933018376531230968850097171, 310980248933033048460008414661464720290, 15877668070149629036659108462311592177, 218974762764962729295419972645475281075, 253812614316520496057264223218271888442, 93451114764188084451355171697895773637, 130861338220461090390975606062765326488, 81687917477427050778113382234386904726, 158746452390538369282393898501548170862, 78058202706228832286977770773932780431, 190315059606649518167674980972153051807, 33805674188359727878168737126085163129, 182957580823589881137421986502306980469, 88590923885820977205710619938104594945, 10700251410404614965331279387955144408, 108882487061600398744012408997329220020, 4582095853261618977696010550335785540, 121098040303334782423046286125942908941, 279130663167323333762154985123000636716, 71595078667724855479252714079142739210, 335788159493811543837746174163985771412, 300941856355841782236443839479859400131, 140035524075322296724530129101199916245, 44979803613051853221826581660666208514, 120761763611626936409733075276014813520, 95781159656530389671574659119525879280, 92225506659246302021456539559689818342, 5717256299873922530505269782215539947, 101474806174689236389173108669718957796, 250126183963922102440838348530947713834, 223466296783489334462782980997445547892, 293510543114622072316045098118727489547, 100646454379336246388038847764751333154, 78073949443391558765829027347804048584, 85188436990130166297669332078973504165, 16769127565839727215701333331961863365, 277763264032632481170308352755320571722, 127683274643063924162885787000297059626, 292907943270850987221084635197224180490, 326893027581818592585739797928578940811, 66937159049283611712880569638785193282, 64313904307970165055492202475012195314, 228599084219930086629068581045840940266, 185820041908942335805143265229115707545, 44530187294733158483053759273861872752, 144679993931332491782710560387800434726, 62387111613495069001637440938396481395, 303015536293312166080430382389044008223, 34191373208550794998229080283758122453, 145700227751912966222656822138181107113, 297940637753734274405788421060898351977, 321255380482946594620822457731890772197, 173524786481898166255564417744032179930, 167760823190905383747115229002708967982, 195068433435090236857869313614457499024, 289127568168963667764110605860331971917, 153500379511443661016530248948555918966, 310279640138367937889069436801918868539, 241326257289710429422683377842498046523, 113899210076217172907794669917242718942, 185533118584713715072195274576432750085, 134875553376024148934671482950262730403, 155508861352494658014541725679312229840, 73261708554132707690591199057536144055, 111188361179273632610408179638532522500, 161132255514980391807564379855102212748, 231280681405116140616693816942517655440, 261923056800660110051483205716172078796, 47846264473605704424035441187137749282, 33557833615841146705470296545926707450, 29382669160695183223850225709660630216, 18836581020703448386483134693148845413, 76767391406035308826175978829305820526, 171175265506587893277617119669661755843, 336532773820821802743509577404149658112, 235693654072577531986078346763543220873, 52945983644056262955784411607878311829, 222347907560399719745401827835468456400, 124787300952878462754014649298034616518, 70128717106131419875192057992864458569, 257726741460968478057570896128737107882, 89954300154799363831276338532880667919, 66543244445797393484053596803336574614, 154801316021819029727015068583374444871, 87661028536931539670709365453451044094, 232074124987351348330205984592175661557, 90351296667891700366483364899975822559, 4140169034972988166464935981847478170, 332259407390806906251229425230440005902, 24248136371461316068550877809758031092, 154477570347904346684960673521004956646, 148052184971879569996558755423802026919, 172372942955740382037692258230963213020, 58960019866848470151017225113303483, 10085301995503894945505503188497002097, 263986944840551561718948526633457353879, 52620663418452914088408357793194565380, 192962728987450001099704065401867959576, 192216177815815960133456692069305652189, 4354038944663820179598581519635978232, 103100836744868021617286162160642189551, 139665145644670175143925676704464416883, 172107857922346866465017536885184182538, 275595062636853940047135592615067994367, 176602674244044460024777020146272391030, 182126751580233033948308649925852488717, 306806492490388131693383287886339939824, 329409827585298612798633523846181281789, 169227753881098025161451429226813079479, 118344572664711625929903232065851751463, 155906203092595207574059255382794434751, 18670115403753017618186719801947682966, 185256573825764803032436835817590143418, 101932275714489267165307289468083750248, 115460735266010897513611577327665277516, 3318053932886715852930440986566436220, 236434063895640641369036579123494975013, 42215541576898924934069686084989557533, 282199905912180022018171818040840470465, 234625047794242433422972338970887751952, 74466706480152281585889115735677968393, 182843129750687100817387487483025627998, 167993966480335461090324425171709338522, 93914097208126046339096567256203751236, 37438215069179813790101117542460679578, 186763529210567429226785771887095211631, 20906051579010739077777960677717617924, 118105218011078668140524904589623759660, 76224004115672024227173794285997317437, 197970234936047730814013201656253619706, 136993930503081508431944926053778941742, 59596106922738578701146480377640361331, 278532526587435997190606624982697905513, 201049664586227944212073552920853136103, 142039833296288659767825469793360747339, 14712327585754134875988207726757471278, 232361427872604963214716709662759731578, 133236469761925060889443830883378565088, 261115739240657016219477600358811934945, 250358484130242104276652120305541742388, 111145438171668630497751042212592299790, 322490793016049082036699234531320965847, 269174611771814203841771567178334372179, 319203307322639669082891054919348851016, 5263857050579920942851553126953319688, 173585637687610142493323750368355814546, 184315683036383469466348764872281041591, 295243092111792844886974953076211576139, 159714651338832602918084828167632652358, 67352088748172985213316605290355193584, 73321759125110834086920796563589934513, 114757041209494984025930696402901946540, 137754944604744261116190945775037879099, 213517125907253728760849255859578109090, 84777258366632621275401261782937704926, 281752564843478259007353835156924524402, 203868668194344465690542874282506017249, 304168678239560866268542688689794250837, 332769845943262257468505597704460597228, 258760862182353690155353246502296325115, 41820986771297451883618522545480909770, 58941522678313233739482649601400880250, 162076412926236522657776788586009392454, 36289375952994050176939878014452297667, 44344622996966721393190753422724729864, 244018582573676642651937093354583925083, 231036928074855289299424811762496944345, 32482390265521926021622308504029639040, 281825301922007671957166752541462305456, 97754253216486828643527492787244952763, 22187127204528927453741271907365731589, 288764726141540311236028461985468292401, 111206300693116892637388627521006925853, 63362162935373811194518250264499075328, 186048642288398041748012246517807743013, 209665358118262695896492891220289050451, 324991070597576940942544260443143359299, 184633033082115229005516980738584089480, 127959915960504006767438816485844431289, 58284168554380500027708921852279604958, 268616655522351778585734452811535474650, 203923556090500063321516706622612945108, 110720509704738690583086814091611461929, 208263174552580999183987057569453471510, 58795929838126158241459964173765705965, 58189016364852771775773774633268727435, 314288315167169904729905940589108611191, 99193185899070702243048681731377180660, 146395439245310328593179880796399332079, 306937664635671020322505509587379619195, 173729261755107569962624763044880719167, 13687015600960442974770888527129607497, 15630383238481178648534607605946566403, 206206238565985320090674499621742351502, 144032059598982086683449762041573199135, 47767354234184232508708220787902545455, 291653426379339802045302095344585660479, 147661904388213997449830487239356828599, 291285529734887524986636625995160303726, 219044159340232732809089271810627215192, 258702545966016979582621514072322033350, 166760836614119067679305949542439640902, 81449336846579840756066534814830732327, 69785025292196834775785304296834009443, 118951044506787551569633661312480386138, 261595027799912244588841954778211507105, 178250305083594092235966835733266471144, 236664726143259884528953837011435074290, 178126006066132918630045412529813615529, 270004281052258074728622232548190478221, 57161029029632548743105968064209436414, 38351336793963265351921098594867345471, 268678157464582600089828747983430184667, 242730789041989560737195181534802011644, 72076692447544516677228995409406094182, 147676449895454663995676664748614442887, 337295428528300933392297337281097488913, 97173856383592476921700405509727121715, 150006950876588419900109730052478078566, 51017968170179556917960667434899302339, 48738519707015215874666176931330409988, 100598223592868459751050773802461882966, 88106952750635219018768645891883118727, 158633092254678142593173472809527711880, 254624694891904237103791243393473439535, 286144728687281580455748764331087794435, 69883128504666604093500956152164035356, 98147177918318317149781361973318591011, 175774225196821025649518300974445558512, 236486798848066559076773692937984038709, 1372016501642904128764891183018633315, 198171139657756683275905734007319062807, 32490230681553074341207704469412055804, 15724994323846935680436487718459339383, 252856407276769039636908966061110674219, 53215689320240851217540481256241435250, 183916925644661072723933598312328385347, 295730185419787963350010913503176955891, 327719298231994695865834340656882459861, 44638744914315061717604073825192429742, 55268504324180056483826509818360438789, 118089351717540633933750611586504174175, 26405634661700075129326685226226188203, 319541049736897746290002953414289812267, 190326110195105766793108291465364688783, 180858416630945726339338597752390484158, 56392089327312937072037245327022531160, 326733682543326849827436204581126024454, 230134917103588676372083645209762412776, 195529594754233864615390838145682113053, 291693080395474718645467785078529609147, 208824499879149137579704737478435706047, 336159372159065289219023330592496983238, 152666931087802238805188583584430090048, 221449593301650700488447635357958698505, 33524569551511385752807554779083838217, 143564884181835655393267014941891731455, 302277404659713948670323568351144285706, 234850865492421573733286595231044069562, 157987406988483235454910863697940511169, 211147225356107847001093852188898336685, 214400993399017357629683139609904620511, 238467707709080833938387954516937267, 163835953367510304189495741171277436805, 211277850544363470576822777849301860968, 190259587969074106946714003416585410602, 58050111264603867063417225818965101742, 281707936918603749042799068634288676946, 309010424013415153064558436432876692112, 55544181722802301316466164517701813835, 281022475119474338379497730309305164291, 147697883781449274189087362876492872608, 335581742081254170483110135890083621819, 222646970826409496340332907372120846589, 132710570933992264729859885392932389819, 261392083902651250725448809207733326559, 148976682726890500999394227917612044525, 220517942369357910112718348490133390906, 325959993762090263294917957203523327551, 51533014543482298762678562152056765419, 187517448929346243091103832280371380461, 43545989095920839174628414445967002031, 89741624348357554764392350827506372571, 22350202558965382700616410282030686992, 197427037606495467352814330734180717652, 326516781187903860812156754385463240591, 24637317085886587119032420617687398777, 72685281647517851472992388025156255167, 114501047244098162717108290256814452018, 130042356743747185042124742105441486416, 83050256183420983155920513451728827083, 45193576065759317119968326881857601100, 246187801798859768593766178879674992602, 195850140060218067401208058606511819881, 317857752186463522141548480068658900118, 213461248999000819181713663015895360647, 291978637686764676281504040433316127271, 116363771430837818757150056560734278563, 63628078494340248148061440680473200540, 322318508224991837885462046464487645167, 310559093142514539417404302164122455116, 340097109995089986808448983869321766608, 272166235224681062284391005105296575634, 22440142002776887085799330928620996442, 266999999500605424420848279932169566610, 80797190097932803160958428654125252019, 263216835480271358691756685179482029610, 17911134651181529495056835779297308390, 271084567077129875187691182617061218222, 32259209617817850990707293078048934661, 113479561328452734676285988401929489933, 242550124540631800551043953017209652817, 98355398099133186880403216684191189841, 228323760051694596372321697514089562151, 283822167319337501903143906754004701909, 62874953873304052882948235237880582871, 25080667408357037410436594406752775699, 74058741495744450334659302829613768059, 62052040854518920417505975099165861860, 26601242316819546950241567631876826311, 98577205921095021630143291948701110942, 97972023052206120934598003526486253417, 224401788201127336470862821257136702682, 324087890084650071333175464210343924148, 291793942027099667433281318571767950321, 178558287811144047214400307868613586044, 121343899042652350429564899545425469971, 311415785232580093944895848537868971668, 191941723477819688298408380161487556178, 1070152214017236726434197116840492917, 243831942864228493593279702230662232540, 296371402340979987965023613004002844613, 298780980051773961954428620614430115371, 64765930515329954832696476481581098319, 306523304096579747447094472062345908551, 62884599979963843854402409762051261062, 20411883775787755222535160555251714841, 83767062860316108232170688358021204815, 240508504833700117726547325816995027930, 299831523554554369048212574769835982155, 63166876563026353184290652622932944573, 49232729039405946164581682985594138825, 76135438860239465347863708427700098903, 24830290197346574302098815381552006049, 50613173837140527464991321050717582856, 259483616262869204208718420968711079054, 133574020709262662165953917050104150539, 54502062725528144051530256325118921534, 83928021588353548164049888283744649102, 184080222874627419306943903405666944053, 162472188833159124987382990707283411466, 306712985964646038098333479218971567438, 104780468980775647312475944228141322903, 315219140591826479122106831509005061685, 203760694092921533040559818346618575763, 328000002175310965484389097494886614918, 240126277292824172292473020390598442708, 72806592116632851066403216599352927397, 142816848122656347891616720396877128120, 281144730550050790198569648657557286609, 25081300560097995280120900541841640549, 25974793183270530115162618683481364577, 104837621652555505910558633555083483637, 319068459194322022885646915613295565028, 102001746610073805944135958379205087981, 24629423098229829810026413257289035916, 228442556175581705898031015482724498872, 487838510616354866557549749085497871, 152535475068410216147621928697098824477, 161394479368968519409066287729247138736, 127790227187195929326149339142938800094, 151984285785280800119676068261136985266, 143533960379430882009483280876804471211, 81364177658856959713186999601306696681, 66681827640036288541212667318589333386, 162550235114272846075588567505545529381, 100260296400115373670562078324923344166, 309034692002549180018020962053126155216, 252317963481978365209163703267508772143, 254695039056104272033760998707426884544, 37042134316236530894173364281097421195, 321416769324514622810901352931414427348, 15325885150963255229177461937547991824, 205992740979925983281861507182573550680, 338805566593057465355684131238217638276, 298011844184510582398838331121911342899, 86582989109310910112168888271790229567, 159150415474902679538610918428489988229, 155520646885040148770604380350142211044, 223568663543415581223383980524030990652, 37996889438376151727851621684510709488, 231184123175943969997338718892734911837, 299630211125105474337811151425902987465, 71622257628808994447187829700551922186, 225312470898087979445721869231756134619, 280375956834163346144345041996030827860, 175643912807410925875169730632460636554, 333274604689165319759690291087259849426, 103610278902204662032138330805734183287, 3261310055243405688716974358605180823, 97151280372271027020087696925417192343, 131645956361081995536021568302670451055, 40859225871003831439728447656727198030, 130680350669562846409206433020030264868, 266471785650233325111432919837529913089, 215632483837923447393637485281128747569, 20353993118007668990881224349066223968, 324245242452993469213632530799806278475, 192945374420217496359545097492472887109, 160728749555005678913625753780483700939, 76640085396825763149673098710769831616, 185209581824321816648792026438022027107, 225248181492337028659611934759049433403, 240800890758661130835798492390288680858, 48838071099394927781198605505089916008, 87771113461465512880444786769005883208, 16626143990583031284818416597403091029, 283426933930280489321831676246055960214, 106766127631067789626070048663492740798, 100229610423077123220834726311963251765, 69961385804105255732309191189718647919, 130224644376009116717476043112010914048, 189292461213507267679177753838263315723, 50183367644402465793318697650558968523, 79539764495828925711665637100912254590, 160520290947648360056167398357304630866, 216503896644095644296262850177497075978, 190088775002783542477674901649245587092, 78909012398999746439318549212821877877, 228439175787681712990999166070461039279, 327255415359523046121976536414081885608, 197660661043679067543987967285412227264, 226844120831540974172174869250206948631, 197332778028627828626728350509274691114, 267360831627904750595728960445795921421, 117157947029152352070292666390725289454, 301737160985041171098305172556348217242, 221051441367151921693948549924634447072, 113558666471775918907700123114319323466, 205980537348269677702800308254306741233, 144356021133256912095200746689949101197, 190529187496662655138594178718103989482, 42954425393878496131255061951348175688, 126721571310865971232403550446770344264, 206156917741968412557311259844393087777, 84493465686728783796936971873733883623, 31366862384503492426398317364571334115, 206679466906148443257335027196299311377, 122081864963987463688649842861064810756, 75034344660009258701501602247599167298, 45229929309850512232403919877943966278, 74972753309990510695985006034435730861, 119687192136004663245902467965467065728, 60358837832650903575239142102382271377, 217876088028046280245690883037155844048, 28686521475096679859803643205008361763, 51146920549218484784776450060916362540, 221292985404143641758397018073661215268, 216096634218093657276484577974371212499, 261200660383947667541997918026534755522, 145291393614858454690037141305390635389, 67714347932430299431419672310647700682, 77794032110684729048756311983753484735, 307639945657180512296970198589218881694, 261012162545896522361419995077359110678, 4322551390793358736500892476491704433, 203137890780900472922452799746043850240, 262891822569961444405499583755759353689, 128697989888247527102959322421145356076, 151090589898431898028242012468534014827, 217148125632904405983473428262353844114, 18581986322050859583786192459221573247, 90492007851420766626884449993013780766, 241118697580403943680281894629444437853, 112025905712149279134747083952404663994, 318234417474587304463729912931739354488, 145146440626202352388088890003923581180, 285289380799101899308099459110266097461, 138131556645687925056473002516579428297, 72034773051422509706217956979214283808, 292921266992939787215573768417116854013, 100745953235534190754792363463100118354, 13094731981268418960027711417399647850, 290150343957420702371331336803118995359, 329549213781672443159497174301887945133, 194684759613766810565450637304290699067, 332385993645498394555640040340405551096, 49003732360914561427653722378959037144, 203718961587941539562582158864519194369, 232479238197699684139248997317656666138, 39743878493751103301909102724059144842, 257536039398220518838914703825426039044, 145083202248469634209105436885872609900, 215149219960433751462255923158566522567, 96330229076126573939074147824899002821, 12751794922618707059702884120704480135, 165012124852236082690054268523743634463, 206607260547929485961365871720928957821, 24210906435014348539467115001309405396, 9167173655218944083766923580574764619, 103184506913311515138175110531388085371, 247189934770895624675060693938152924356, 281318085141874839960564638092123909427, 608506990628614017815229337024392154, 141221116081365220165117703239369402158, 112070024345653582926705682508112507836, 106100187737539357946823967698135554562, 124579321535682577380255845582941536901, 8686000004180262932937732513117276836, 295952401463854004556367931890418782298, 11955308321554217606949099807418628505, 106788672369518489738680164253114820339, 130957638458631747909582226232496913373, 154662879783116049258211217702524097388, 8436249489148674521342431828790034088, 32604974522399041090920294448249960968, 39015418000435588907979253715260665336, 212239922796546346633091698965104358739, 317536942699368262279303681823690613550, 70301361290539224709084152633738352536, 152456481060936738837754813620729945405, 49911928536408006656606544503773443476, 231375604976689343948410563440208575846, 192301998739693095378094491932763301450, 333122866681781504249049159010800725288, 68890280898521926580335116701224215209, 132512539160569802769092197900920272866, 107480916331632684787843652660523439716, 210665653695495011589161885308139533193, 325001341489073933883866531060716503941, 94769968469708706738516760399685340667, 177934963984673994312568862934784724255, 125282944012087879228670670686177427515, 8801745414353509000352491635220559478, 97302251952509836307971094276302338223, 168725876377825849678327605004388381390, 339969967587134920095315640669584009395, 317849588945674108071292584114252418930, 194887844411944802252011033380628858502, 161755881231092195088225321556208950885, 134136686088902922930572761306221534167, 293339075626489659981104058860711403008, 307428344692953298220211699805540869857, 99781879308353035106359064056108992559, 19523778556606559145021832721084537339, 183904572639743601439337101653588389711, 41970940195753611178566089188820994424, 24759071845532622397704605509077909848, 27110517529572603721015134864739536668, 8940250022500346315373006890880825787, 331745356853893391685484601608196343617, 191001871243341847418155323341992808870, 200868503246703582092469665065642622308, 179253829655505987519652499164571564033, 84439460446339488735751393384485387590, 82782953588874819298367900283699197920, 99645855867870009698519197134382478750, 90631436413541750869619923689524018221, 115191496228743726404404087928047488425, 240905752217557748703250400978221924567, 217741357580741214498355113176998395026, 68027829224296035025624796941933555325, 82462268786166240586189837578086755380, 181006473341438932532753814945005375747, 219834466796479163394358036113612210649, 326087856772974850641739907563354261269, 136517055387218967611820184494862775559, 7132844783582600436333250311100451488, 163190220406226818782520995310095502740, 138666686834537265013952465679898905480, 38154071453176727757367745548048659200, 48141141234192935227740667227965944672, 227793801448393639688030822779567245604, 176480194451195023198982036347991649070, 157688013766539885468639390643757149567, 249909175003899947073331688221960791021, 235776866355381995393661448404437593528, 5054336945857982655085692397493072412, 9159657324535943692931225915386977603, 24381859265650667119249892090536128564, 171121842493728871107646157539784687627, 188912908488843303667738292137864442086, 46135417028489807298520182793221599579, 118838648401402237370572655623158679530, 93915986758419882151670543844316577197, 26083264710680472735226512049742064179, 27534869689633894141860540926173194262, 19947673019942976055624968388601730756, 246843224447660913574838356000418015737, 43832565862042770834987850237509845488, 85613115924084187244486862750874750818, 147923109554724966246166326032890303215, 161089900992189501085601881030789278085, 283365442183034943803622393642881776613, 237409336202526758038463301101582267847, 153135720118365627206071644432976413268, 169418500323121250488975228040762473895, 317601377940365576993383699799928835791, 77204042744488472456100862083120825456, 78489315429300956803021066024227429159, 141774790172381965372555354643426228375, 171408977310270498104295074205325700329, 61581681169660335726054772671811107546, 78591591413842605844337915594380049603, 194372116624837599331391530909830764258, 167557909499167212850434093272288859452, 205918786347373082445470698352866415937, 53219102829916214734178766120488196262, 92248980120613862955249075593968487074, 201665911330748094036374877117825873770, 172333077996100941056375782290110671953, 10639404711934680239793103984427697218, 129270558915282741200786811055595168623, 66671106874320953195945527896322218494, 56362538851565529532777918708215663163, 218755891932277802487443011488296110419, 282707264539494119527983887194380582049, 257828994343128261356248426119304229212, 118981770888408953585031528971414589665, 147055629285626963240164699118015634288, 149248586283956585677079990773429075674, 141034291479972387520394449800227904439, 177706426548048535575476328013600368344, 185467498945592003041743768663006727260, 284353571480003494088476311240292902524, 129157884193548918322246807935905761349, 45958033290120572301504354348155818831, 187319845368782295218324826839346915478, 232174419025485907212290633323880496261, 173208848733694007305110164484413892717, 330378506553487828022289243138623613847, 128600496876521825428907699578870429363, 309377713589480423016088414292949281919, 54533742410392593219911335466225835942, 175118696400146088538325259559267588070, 328410824565655856115082218040526102569, 332301530361498315773181219798943516158, 57487435170882640537102507691882025756, 105443911890895824053364300995659284136, 35439124141364642825331834299499244586, 127569650650520089950336884754169040633, 238475769948968038413088347128543685019, 340230686314278639644670519993564652785, 137524342324272639792128667865529162798, 34489142660206192244638179512728210919, 14807327535801208720939462143935794000, 144210497511106968851360776455183262109, 321862797020597165660239084887200818547, 160120506662766281367629363382740141812, 222840259099282862959628171256448565630, 280366616759183479189160143079461158223, 254016292955739519056556915838301784399, 125099369469328899297968306272318495183, 245696990216331703289480424953312082984, 23988966933944637997756780939851525866, 92141889108221316241473580105237390130, 293597646663381660126075017349965242311, 282366740517121419992570380357867410984, 28019669051269363773098596723535603397, 18350208607857383774175448907042972000, 26281971580277019559501385893108048414, 151998590867691422900527548445261474324, 314581807658231702426275889044947792941, 11802527022664086861094726294750309746, 212920107203831560371408442246121847044, 42155294866689189388476578622899800060, 77618681410462126362370854158957543813, 331915143619054730897824403869948260132, 9154575918893668687975363742884548862, 128578064216645454177574950519161591796, 246692174925078580967536473363746821897, 278200718343605241999852729195589671367, 289532213618926621909906401041315947945, 139020131030623335080245284284952745395, 204078293940635746952644048684572709266, 212582431634379293027043473356470591650, 83277242817631560242832745364096497936, 139908032862394043401871828370608520491, 233283365785109721299125248414302743367, 45180050084862773798278510117489781289, 151028063788718880160911523723185215621, 284165688053528505071413932806079571460, 75115200135084408826103295825274470507, 317841471106135601424968447642041456751, 330936027437462312639115488015889874272, 11400974308724135549541272438706189489, 98194147101098615419817353901307822686, 19083903680662942632921710664348266635, 120111073987446458728716290698136167300, 7085091040210477101849684769348477127, 89598181449334209823028529730230534268, 272370193104415359975633069793993319353, 219657798896768163954349475975576879418, 7610093770063166187823138228297568859, 186277274678226429933640623388883686551, 280337138192733967925605465258277929708, 280305464710016925777418930018332101680, 76939475910966523995626298230906302307, 89583830910800284389898015023572500354, 214494989137268676447620845624860093058, 282042701769189053448502309103084807678, 195319550633050507592657846957756835854, 101298089844781424242909389884267549703, 57323383972067335882248116952412863700, 198662282425620960895288961165225501053, 169829017251731062414038290758896005708, 142214183640987397619763630174211493406, 242921481954221355347357635389628952752, 86444074411375409113860727815298681761, 241705817500088764919160918048622124814, 101996136402854941397154598829855082244, 308119349010914166901089752803586666747, 146780157593823431532632581699589852527, 176342164229079156554880111659337752020, 284464067583342709560882350976852638191, 318520896052905537939643171744179696800, 309692362491358367795826869530740353780, 81883331442805940824190072243230201769, 40461002654980010191530556859776489908, 143568286096558465342639880581736191195, 287902901408746243960777622311267705288, 192755626946609613130951001481379336864, 19185173598772231912255851687302057689, 306596513572568841707105401406287288332, 14946152436868493990863176022651316886, 14962745348217758083757311344227942212, 98762559219125385969250566555626636228, 88452046835339743276743969080660883051, 255006788538571426282302785331244096160, 298133244338027759515676126307994573791, 289713680561391934104600903082513195834, 6509198084294133477783383531430502908, 134151974732885956753740011637763040246, 87362570726128056776970220939369810224, 242340849782089386649324646540944689059, 80311968205754634925252351691014340934, 296639728272405995109878768228707459877, 239878228412217155153225339620141105761, 328481953499066285368825721083887411393, 267638959072193818192817729472809937758, 55901010754279724627446951880160843494, 134120570852102063794710414975895331341, 268282101598171858305904979728110439969, 133115475052362841153922902932560503647, 87420802201444622047637854287829318638, 82396439451932987087447125552739384318, 1876202374807614587045565489365430642, 234663379325820585633162355958110474729, 10477438521373793988424548076128434793, 51575493669184214220835024953705353879, 294202418541653852113566944693524169972, 243065006438136995084268867931350520897, 165563281347488286544679207225047121516, 199256259617987774322048371409462339716, 38181415177061924997427632750333839704, 338621781134776014707752891609922438875, 35191514456503305899147695771937686303, 255740503149235404595618800963524331318, 210159023723254073773170195848698346842, 50815445925988658366300566568106143873, 130507553479268178022445324403660280591, 188105299670926131458163097103866129686, 318341190890997707679931845202937906404, 333167289834033941247086087296477735171, 31152292965806541243471644960164803402, 322959149589159822677083296363041680620, 145432027029810845431911495616840273136, 289586612819265064692254929052795333943, 321016321498242346319085250882953071317, 333774247243154139728033066657483265709, 73857088520153259038082252405832765950, 168172954865784502453913275421741703410, 219858405214732954261217904225620806823, 166157296345573107823072648447735500710, 239117388366827515987894603868941334335, 337565265896303781374613964074732673748, 293979297612711306961930751088185002698, 302290265639523636230612520601430893623, 171930197394677019014162907945540532977, 33063706322487903548067488188112553088, 265176217739855241251787296281924950819, 52741743155186480691579822845271807180, 138223652320851990774091563576126056682, 165413849247693053164143795710349076456, 179906656514831901924289731030458164062, 65621901982134447110616463734962887319, 96578506578370457914522667865639397987, 144948402496247290087747436980217789738, 242061453856137576579379848938877771197, 52992268341174462052137298580720330539, 198914209540765966571243327715176405309, 32850765978393773591565539530428878086, 299241510450506304473728138214182804146, 312472182890872548527934654803274966221, 69802744866645079800490741264595693492, 64839826350579396723958404788485861762, 262037956153476052799867102378704329783, 81088313392345927195069403310626026255, 184337961816240717612428981857529814327, 334397681177429684919070796945429570829, 163770189377529170242947612280458066447, 85162750321923885380234576520248166732, 204238175869181827954860181105079274890, 248977358915661169618954536358388972282, 19128211135788689250976454451640944260, 120015932042415875852049607347116465249, 20445943071299787803996201834492218754, 311720661768656532842943423958513963139, 161179082088575556882494808531116345037, 229810086297415324315718034779279060297, 27079875488738278709323872753588308029, 51596961656436809427947128818338463000, 295340149630257512900837977205704086525, 242836600226621361249497493444877603700, 330353616730000615862244559416810133910, 178453087972633564332491822095524426346, 335712339267118417008069462237086700951, 140327809354524572656303794794768168409, 254766150750089431268984836943497627580, 321429157197430692829154205798343159125, 203478990800241585526279017944450485122, 126101943020157393768212800648166303668, 138442767387839906492872873788000143256, 238999219070950390579806751955400133282, 181805075450758631277121661413736809544, 263167628894445070681384321218096310352, 242459251824864626618524434419256623897, 72886068445472866271302863840235082959, 124875407270094312730242138481128189139, 234543872740617671179573204895240712947, 201405892915986840741386085754931928602, 756204236894991393587949832674395892, 290297346584206022813938742932783672501, 20694379877058927985524132464502010674, 10427887775599319773582857300643406244, 123248713150903218052831515644638749027, 339372186985207707838639994173235775761, 44520834520998717874344077756310405975, 288058312104000081376184680712643799050, 239952070788953906367423901696060696153, 96290461109881573665288762273140309029, 19745716214680700796864027833403619990, 105662523623117875039271238696548287070, 229422777810609897034285195939464476268, 165366600830893504451146949071209304351, 61295677950512867011912388810224891112, 241769465370637574878821330615974599127, 221193764785421392465609777096885438179, 32737843735434455103597778941328182409, 234421850930692365340589569948266931403, 334974752756023876043565343015166505727, 174902199319461979771349564835261806395, 54552701472907430433459072618661882916, 252333327117288367094790132346888070217, 324200823809995735797469736453965934609, 121350752622316553440028783752564948513, 225213593295886381403514684901807739579, 57559330795212695733817544207605269283, 315878474452786044900033866554013018869, 140891658190857436897621152875359949136, 136699067387947036734578212358689709586, 82030483879220218030162885135657232304, 334085713936741875187392420550999849593, 255930388584822709851725741936366228298, 256852128960462388551661336218723385645, 30263575381668297213429703129903897309, 238770356746006580102793091423410571196, 260300793222212111367832021554471005317, 340126067007917353057074584779986389465, 21516009984236416260273804451137842875, 219970273246485283572837609831465972516, 90295622209692962130998425598621796666, 130233848376558887933543559069875788617, 273498606112064368727867571963348925240, 7867889217078573712451852953098782988, 318613304692441636244109493513153108526, 167999169073386088065466060597052567343, 273921262923383518939020042783817780617, 197464263734818037907813441793045561180, 84912511082685363473336600954485095508, 288964571564520745331374287048264797534, 38423602384112544723076979137894903095, 97215857243952589578605341141801809344, 203349962885871980735820155439852994357, 251614440138480765773840640589274010482, 162282489500685982095464261841649467181, 115593800935570659698547193332052324495, 258283503810112951442845568068895680804, 144058902715477976159132102053925209861, 310451184532067127104853445740124270659, 315745959279966159286260768741263949182, 112362078895234195072733528358935515094, 225731603283431331327072271594541313714, 108505303185197537653223510788523049268, 326791888437214065407706280153451173778, 59745851807033199866213343535451343097, 334059579012654235006688577159582052187, 9105699099050566989567843189977293506, 132675244194742947016463503035472045186, 304841308833312769859101874371915785849, 119261887367363387437470841163609097281, 155513194367249776891119993447596987263, 247046098309422457466796890661244471858, 84047935576057681111256766780575807687, 207758947376979815649997352157450886018, 211001931996899423174427699493258823825, 130194141216110980886860065882189652672, 168841844430086464105842605176240783351, 102285971013970284328389943289254222033, 95800933622283230843444543378315130744, 56450884382507945408365983123394109105, 101322667289931419645570665078905803964, 131567635545354996226945317237890531063, 128053044239988205152346895938364042692, 212473433080701161739842242784794673757, 220934581980905488540852916145482840337, 78687133428624389605315770499890156201, 275534760849983406616029858809672386314, 318182171031430065317266016934203261907, 200490416199236571491385131025727695858, 173997629087914840082951686279803335041, 119038413657970714365093509514208620320, 111870904809039957987582495622013336911, 26871683144342766650666747505933549511, 248716829673829669460606243816088003442, 177014687315413212014606652402866510663, 273916921123125076769483816818065501951, 61441164529938139842293962998492967264, 277134567749949614470454526930539758030, 255275567538658431963639808999111627087, 219190985004968985962133643557967955119, 237969166689681875326775362033812320911, 54740103333580720529518948674342732019, 38238495391454563206298195881690186571, 243106936353378572943968826731415345902, 181358243442203405918731430705007148770, 171625940752705227812756541502277934321, 149224320118718612506861181718307570238, 93666920529073077209719149353983569876, 202259156970076642842765947539273831007, 177279997184749677571186625280068144037, 269188490806131297214924539069623062968, 172586493417343267061096413969065207885, 138008283282020667311286342384693791604, 25639983140353133505503299379797803512, 240282233637158009386352072456135100764, 47059545240329489143191840496081447725, 221675214736531574550417530339822069166, 271128593568871381421428658602999256260, 33903651646055432701126088042396431329, 44399211729230799673326075336016668879, 87493355092098430365902674037245718725, 141655734039807332552754817132599513630, 132949620533798666778571146319023227652, 120235304745291734797495455128318393741, 196973136581191377055661484000654804498, 246721795804802285118857190015090351015, 173478050760019161658152786222806938600, 4217871163174545953433057406475309356, 335457302298133390566066569226301909499, 255410797245953167775697481230569782202, 33727767878827990336693191026421851864, 88069503622302796231195486200403029560, 245778347692362831318776423355082288698, 120740906346633392092152012324805135257, 292166451563203737355175330721190474160, 335501264672737516436238792455086528396, 171054273255801643513873384210410902841, 51631913581061140589728251310235032722, 644550410435374324486791231219091008, 111100460655599990569876939194804636519, 207586746031999221609048402703054904295, 308807058503655036890824110104571653949, 154733241277823643295340228779050234721, 186044989824954788686249249311719822717, 306726990789946451964985796989041997461, 271292122504199211181515183489521194229, 112663849203359049666819488184447709807, 232449262411578322969161983251083637804, 115393513365250930033765264367092290059, 6151759139856016327206606250121010232, 206460124131661226308712601452777430599, 17442485772929691088451979182564929340, 319378242377500440277580175899829558612, 277873407942677682176362496747572526994, 297664162243095667390513580044405388362, 161755980911228008659373862758225874621, 54846688089054121224820821229866060071, 225976809079946317578744923778990373211, 218894619877719861365697048260078103111, 170292939788097061042972627906664031748, 235989642421850111345713433770045868089, 171099004692859401037752666013717213053, 79852328710237499599937437880875659719, 121255837575966296732885692701337194826, 158666949500807834177424464959592182780, 151335333351934736626634314633125967346, 154491207813884178849083244753495377926, 133750129486561540870154170777099571728, 261124035216342835905702995738170667400, 5434100330694859752311055245973196566, 255481855936761000767221146960359886027, 299225314684550797379281540471202381449, 76939415707933852870025904648366607094, 49459770870787391307951490673243512812, 264296652226453316732930191892459074476, 220343074732742317043663715254048718996, 286155695441966036026527278052486536831, 58017145176011783006952717640189833364, 78295460352407284713207914719920686861, 234529670213333217024846749060463509410, 308557480798804964304844476938673020765, 31794661586493843278851401333395437013, 283400249548700151439650054353616602733, 3418520602686317291093323907684051853, 54379998792375378693400567354743003367, 13596530376050162259392248746615029337, 263755817947449403003695232065777802118, 240021701849846029882066327273693868715, 28067770006956314792365508414138650974, 197876459844173542589712622948342986954, 109666263941749691640388530623038593221, 308869561301185822686049869924118045675, 270754053590809247037484517214746322900, 1594333804460004476125141297570243922, 124039992589641388538335113250593823975, 1859002768028635779133576126716086396, 72094358444536243943919714238734442214, 227256461398145456814636450504109490586, 129557067661401134973530632782471426553, 12756433598306803458553432046982849991, 142841281651426763658983884316844118484, 126532576257113427232931408690773230789, 82448364650538551178821019335291358854, 122169708863738202876691937589337281411, 38326027468974705511592866629303027398, 60801029698288222000958533684338427364, 9938813275880402915004401625583768715, 153011537376533627241443185322752277975, 89915352966865598298126638893427589661, 72555604279881516031135774951798535636, 146230556511021861996150496495941300279, 86392261898413540982653729558132202292, 190260867246739039923289535567729514370, 61739298397908541223318739236092644870, 70387213394541769447009125181171561833, 254568322201828865172800045020066812674, 155331273340017861087358192663117098252, 243267331825513297180797369029721853983, 50901307056564574202414132981800539101, 319752746081503789210212598607111921474, 305348624896769161025533934175940044028, 44586427854493435501888822731524935096, 23139198546268887695626731873948564369, 1580350450596084779819805223850142741, 58598313592897678928224749467915353739, 186628886765869746424691179471342857054, 26007546983926886329561401025332392553, 136866061282865219009769368550274992661, 102379378860813716268613416722972677446, 285356770920323486989944272107500587295, 12881564216062828152935900864704734640, 5738255391532573689476425928557053963, 245378420114882270745282215543498436424, 289290639466020113389152191130639922364, 197088820735709596246881271970208686094, 63582193445490207897728316724603436641, 294750164780353278705688976194632058670, 91672969134080443667212087286432897041, 283139295469925701846569863948988537652, 84427238221266725219333688778560251856, 238430341232072757541780559289509678040, 241657273663977573237233809333098835327, 284977751296093911064910117635780249877, 268672842633709226054897317182045356467, 281602108442562406584965028121783114905, 336271000957025503031154489668901696585, 58081479489912475995788435283917283907, 3354566357744869669938771929279814629, 241698000887777380826897042173626276354, 328640551370917716004899046048712883900, 337947294309494144654965323296829027512, 285459287113436968715637835714493384769, 127306009114591450703581598696988844153, 114251897043262183498526726964766757652, 204989795455403843215313542934309233076, 88923803809465684440660743924124096835, 276628121147460795358536575420440815413, 129151837137325991880953566061996313785, 308712875460079207959328947126939369642, 223919372394334472461079971086062655573, 6496694045263372884611098162475311400, 309208661747866933303167793796708518465, 117019379012670166086893506973299006790, 314647852815271984521237319015114283626, 145671895906069566247665494521908485401, 79285159216904721937456610803308009838, 104306039968980049487886983469360621870, 271144118218777764355821993540644897654, 142013166515902324511695725756224335551, 69712101256826376512145183392026293013, 289011344490896586628353482138664013894, 303533570870900940479206355511055680200, 333371316575961748227424185807179880527, 279397341696313686537584112076686197492, 280604718092611904079659925170142420436, 157955158666398283040519046325503687976, 26386194927269267426068863069494626582, 76362525163506633342466423912159371103, 1120820225674751239562177712489900033, 303652304259759878396115604039200457419, 339728290747723279582182995906186038255, 214187394102043948329736693059573434476, 278933326104905205981480903950901203364, 162298724303815966603493230014085004789, 174557245207867798355437691047484064249, 126025533603736948518437080813852398369, 235604583483234050175545797234442635443, 315738075460768158835040904333416308667, 165600008773210493734163241577432526810, 229413658663262707163800848514652018356, 180400946040341464179224465836944728732, 53697976830871470972066243277346824515, 246245158080151707629006677321350195657, 32549049007345367106396704989497628182, 188768923608518511026898705449397858312, 66089241388234252590427410168068937120, 284493645273135358020057589050697514570, 259585760662288974110571127302752790055, 128611073808634991206355911794427857091, 160692350842898831161989031461679306984, 286212766563761564937627989917589503542, 94270748294967033708846213648081433838, 337889268294387135801575085492043143014, 41097124248975060066258980216882512816, 54496624966375219582010412198673853020, 78271167960061720299839463467398425048, 331691473871440842734297131424501795119, 52389108001921297788995438518437309444, 115948240325981825798433623398886865043, 173792405637066825827613180491280689692, 312737504320531964496672756936981012661, 174893340477808012423021185695025433221, 203674034122704446707548943025898713341, 262856129257944526555221685903753540474, 122444878947884926306166469940778296731, 150451689011845830269264198989692368835, 130511818900523083017915190775177931218, 272945577835057361053558650395887888315, 105244931484262871651054473776379295805, 37943191653064897863481852339761230061, 310483137140463139003072405424470428248, 215906560662601893160139249613835644611, 79777912312886913043229536949422046230, 184740075295713526039238852630658960779, 28308315755608394214288182383914705673, 335963893399535344672240784886950197664, 259398217865452600902309038925580623929, 15589521797824690687745085303227778779, 247701893960011569743679573847342099288, 287315910311127363731209174719847944618, 185661081037848846372797145616922885506, 87392880669367822197150117845623926668, 98652323334442834551613199261802961857, 339674911799003024854247482734807223849, 26085343535821549820443505253262746662, 103351227219552118792809034000400380774, 180910755364617965251616471725309369588, 127474170284939954597463737320055321346, 106576176687547066010020367059015632744, 136072359849456819082237903270925452320, 288230314456349837399612166521658547912, 37286202683927847530438927605789991292, 304143116609675162231678789379134044203, 3481032227467534642438664463244646569, 283163447130180583490661402143121461239, 137142187666020450755866525491090890888, 312169342091850856492769938184319377364, 299374894338452700893984884182662049018, 274487268964513505238609977377009469847, 137681041082130178922211470840292953220, 127716887622588671648245172948135800996, 130510770272799509275559514375932270906, 218114281136585882531905894345622580085, 70009730070928245091827735284746842810, 186945495634235312165265340824312372801, 139482447071652165700345552246840106251, 314666886044720341636247958827934437762, 93738818828264242384998587112919792668, 166829964504899129862000725649093254640, 122252281751428379420205085924344268168, 299214573046485600798633613620638992165, 138289120987393626841304270536731491673, 49736639543238773485213249547685243003, 231845654713579667872170122005400762515, 254038986329993482444953318182905643776, 284482432792336283973531767623844314522, 173274863891332492980514700733823821343, 129931492059209075440248481742416356727, 54640218727714771148323759354605277080, 244776508811689945482410922793720734891, 316943739370165658161351267587173388063, 244653604744860290339742117262856022557, 163879292924569844244991447337113720088, 187475089536945391751250193013891648390, 206142964141804791380098501628663275105, 221916107070494847139006354045607359272, 102562686458613075885871084648377274500, 246037610485710072090915683058995325598, 231607597922220233036701583054697650876, 120242303707408032129966142840423500307, 232435799007384991630432909483760792194, 15485806838914089470884161118121533644, 323155208196286105658969469537057873305, 116214545221683233326764715914266156155, 5389533281888946229476146046112715299, 77779798617954354726681232743914007523, 37496488009424267403739927427464626833, 33733108620982221275564714289586868159, 149014184662883995301901422908574179535, 138548258629235132560975930078054814033, 327805457869361808907444316674818763554, 58914946037863150979141473754788281767, 157664303695337765570996496114571443340, 323271111244669948190511346622962966488, 162144458349555051817798881619483939668, 182346510128373435485484528889959887574, 150397737668914937282784647742661681759, 93456647733548590428028753376211810731, 38632956222076174921267846384330416424, 72543541256927519026085641446241045733, 209534758627508400348573766613092845605, 257411823935977254567022933264099680710, 264560197009986728523800236563784649457, 186798434483853578051162347646365030879, 99178993082682427566576624453261050822, 230039820625896280659145390233591148945, 202151613446145966605645372787983323808, 9783007629808042372805173305154696439, 139977769545065557015515292703852700026, 100704864078939051002820852386464186481, 100802971012183186033026057056742795896, 295048797856834311760597838404529895814, 94765220645920160345197956854732566130, 269008127147701966385736022449596755977, 117788698498805248171425559232162908681, 211154479936067353351399201167754393503, 51886765025236505529198425296818000852, 48492722433048987826135916943713530479, 293516831825834756193595056029433583933, 115483118295859388234062906054602392288, 207585939443144569136365649526147541862, 4014792881570985778762400780110810698, 311574880577334155807058948519262578420, 336549590744889214702783922753803063142, 118123261488560369008241159994912617517, 21932724975080384226943265355744160198, 164133266015113080138370977494357610769, 170236891902909359044893360123236726992, 39776543599347541468555609637490201871, 109913024149015525541627898013153568868, 83617986824220417783311093055451326311, 196541392779184357426563447708397268693, 12120566817089399614018120758805565313, 5984702290450717821226801236395500744, 152061127729080434280184403481927467560, 10253400811609779428780313793650608391, 338672339113401663941999947122201159112, 321175714165684812119239883182270191616, 187925714063122985532682019805845681811, 165662030781418990324913782553987442297, 49766835904735207733479994779379702940, 314077158034823883943353670140750966316, 159834736640995013185277027145828108210, 188118812075684887697984831242202460645, 127115609434120510793357828309775689882, 243069478613642662098338233146392446821, 162062460937104373297515950677459200390, 66281919368521728047958181946848379334, 169040631090305615663226992350913375667, 85070368739341314912885147815800441721, 35181036396907268895938596005078061423, 10057409485371709101439539712581490138, 321363935560874516506904661226541128683, 267475454168753645743656714638968068470, 112263282838003252067838493987000391710, 333235402674795006036749739402618446475, 330863475507465983235640242526341321030, 281285110040313357089656915065984246080, 8723662743472265534925997845887626490, 23505545677311878631332145095102947590, 232676051280874885689302471473137444605, 198435649991912895806272757974623856792, 242274146757764706452117540097825430143, 152555236084266730539711423782947816653, 106177912942000250174704490039064191552, 61531772388022270504561650353767160707, 58792017929180626774650482672914093168, 282384793418417196231986492044363335099, 171638371956282109746473569784599856978, 310184552799289010967922413804839215511, 320178163818863796782751571959872066468, 258632813476195535044209951986225191019, 22043718237897839892229538443439169959, 135913486296426275973462099102180085698, 33547361224197750969477342380452810336, 39898277744518340070124078779642207865, 101972248580542269403377386075033911700, 145694479485151341697223002950130403084, 207794622854374292826681501071187972244, 172225427612167707252411403583176974629, 80845069188482119769031608620846316254, 252776525294534128529856297189222891477, 334363635757357031649495915620903020800, 95224646634151805209592705012587389231, 328094129122442580456477294031942150018, 291984436351819257197155942955708774674, 137267200860241778869425904724775015774, 53340447593854411009372934068016668411, 118555668768344232052090692029140692725, 284779495987594541547398889358035284154, 279844374819992714582937926474812304130, 97343145301327315514093276066109227559, 121756048920399792417532140730049867921, 64209588063571456187848643287464203998, 42805848739055830313549990044698663282, 146577705461039164784954030055284901545, 201120943435881313285070447206998115619, 231839563645769853745990293552492759738, 90866615986646835052725344312375093752, 21160327449609558537808670989004632125, 248623645506170578796814706175012913771, 231359013290346158712981397877900173013, 309533967782777319672723513411208981544, 330258149955199097886851631142460983020, 34419587913787589774745317205241463630, 38000103606517795920519992929057555577, 105754840550544528100970663067352851715, 129848377991066642257829367560243160539, 330525803994324880461140743804260332605, 264961265879375411994718814530238903840, 191543170443231586379947381345836023054, 265892486290933987442197672364591918218, 113626009459308840791922656959523415680, 105694308179243554296225593424471964998, 73975235733466088131857886065640081552, 275629123781879479910647701071335385180, 60859105999386218554415748487325684177, 261041549005161127067375885926023160319, 122094133264790864046209743743816190966, 58929299608147867777763912469659054831, 14132607130847434522425647892890323009, 81223833511395514491805483580153982710, 72684019137127002644196309229137290528, 190926749333340794232679696205360149662, 18234852443304452712609990867666377158, 118251349025554167809524321638656326642, 9203904595093872612618901674568736398, 140241355538053287434056238196352169019, 21662476277778351955066033358587029474, 242256217557580814617085654007838416901, 45730813261848423995318758657853807853, 118700799478990399447621652644386150793, 16135088186464334658479390964517278662, 254790271846750027554672424263844559627, 235707305853376900698412274138379606320, 256678584921200733978690288639819924118, 83980963731216160506671196398339418866, 244887913516009319584286505924114612911, 249163133383550323112096374610067146700, 31815946009640694098293713763842440268, 54342271140888328053519042894278530149, 105003210571293029309085415194615597723, 267185497905141873050601865853276014225, 47251574992501707757000782074918130381, 156321803560294674858615154100493560475, 24276670524377100403647608091598200709, 295519851401617306771825998836604842120, 97325774175621764976228895095963571392, 264706390099314154135869233505578263214, 16458200481362852204048021552246425787, 30573017706798681153471813241735669864, 79647735585647018274490489874773879070, 54877093044840180047225498143891668442, 291546129771421749465615735135886144561, 52265020424508186167537951581907299045, 18365993033604130652393638628556128956, 140964274258577242065648855812546889055, 1741041844320011801254025076401424367, 329783818441688716254966508890589119480, 93541451119249525865301977271306026498, 4990787959342585021356562648509980049, 191009235500241572635971497451183763248, 99113191469159535452195470604848178664, 39738633878021796546727576879762331283, 20661075578618816278348497499391132136, 68946177153077349628078599806531559124, 199164999931011809398898831005463138752, 275414428461719753820492713039260097120, 219541574185760854129945293509217793144, 25343216038352396671043746335875284380, 28146649614357413485870909796343115094, 203516335412259707289603725411815598944, 215629845143180786010813998426572690966, 300005845114289094804312579696370083897, 338613168660796974078031370969136798548, 294174137987539049125596236406613771742, 172735694592835849599044738510714814430, 176484895597788833977925801492812055774, 71783918298397030392536881028352901400, 180327575831642975117939415294312092003, 73010005065486082769583508467884848239, 100246568771164116451846546652641324230, 322421318870342979521732855264919080447, 9256935325335701790639551945553403466, 290226572495378207196058007351602326872, 207773609112215737801530149134994314730, 305891940374842285261308799188710605833, 142412110441529159612244634589158377367, 308941241488032424030916644516600871657, 152500844312640891432578675781104174024, 19189457008882037026751180705418323136, 136413552397110893443682518163720471209, 53616024027736989757130767403680954120, 218980293237894307997370737770612303592, 213839856109481103153692304940532412027, 209317314524237190505202536844218889273, 239090596369214936483425975695473242352, 121343902166075829298460951001728846924, 245763732671447601724952318231787613297, 110602248167186159329811266624408012119, 270548133963561329293981795098692587864, 265299344078053283624204720851602992608, 22624609677204091463439316164034629507, 224065605579233370282265299966120148082, 233090631832966099294351482290929820346, 75637051693466450235252503018058774690, 80549810687808257540545458576550803821, 162302746982894558991578270448358960052, 283249910760212893885472931351995129603, 169481696184130191691698107426208632305, 330545703763963668927830136575102703629, 213287516262567727769033991375090147918, 166233480265134934778043820393595739941, 126301641122670988995508000190139132367, 29448789965517409992887542377011041807, 231994571497493885111944857267266531771, 71468139705121303064100307439160225825, 92693055338976199049733848605996075222, 139023812002869613744667655164207071499, 74523937017128238621372614995060011236, 49648936527934899847184748925481697886, 131532756418930299878283379997837387569, 271168875208153217692014820623229373444, 140234129071804361264686751430736559716, 206592226993938210808083881095117081078, 256610611083681762019692078890068904072, 329968223109055008135359897557164238439, 77208646657466171261236225953126819582, 308278598173430636621478684572121386725, 19571512917344973021220290551595203260, 58860357308651927412594079406327652932, 82213107612565583122594782462567660040, 113020839017806548548327169731011787238, 123520071132790774350136619926312504932, 220720636713793622608117707898026829132, 339963536313957764061887758834444822921, 282706213005419013498308858340805831963, 59260700095583485454516999947487348301, 77741396814432386594043925420420341663, 188529151105286424669780838197957330601, 18777144561751514590945769548613165607, 227803460059918893305722066600050309606, 59686354791357251523726430058952684209, 44996616519274588264076941049274462373, 195405166908447487930378227822429245446, 133084134058402448851153675631096661743, 138386747683057428977599816862766871966, 37221841764492300704268982576841763266, 261029184817430850229779504741286372009, 335138246722445304312366598383272922615, 252692921671571774628729988067080143261, 115811073576432070353103310458370374577, 204616249968735455187896202344191874974, 29067403287947147135977202098220447129, 294234462570704462124599108293971676686, 170250486083841508259304405664216650215, 244030041331784695138640948063880930237, 160612483529456746909826374970203085153, 44062839561656817430049724045449825517, 328850923958685976500973319938652235763, 136253066452359779057681505174851425758, 41460080918691957784918231826211010496, 330637174686190003138298732540693654337, 80011474447551985352160327769926580238, 8452901579543508953482322869969947729, 199034076418991557194068106095130806425, 224678028482296577795106409354896644069, 16402294640299856854782334327552214468, 177440002989724535227935582832606494137, 66788755144330818308578628791336040968, 180649130137079078794738921641561194974, 228366969757743241446736212471046471561, 75388633142391937245437682673056450315, 99434511822652247659344008801631639298, 332327350485761195017602172858987486125, 274136695172961945486053842703937791396, 18551362694454486094913511547486912428, 169942117875497880813338681323892888492, 44867560906104516828612327679920730710, 164582222624228811645806181235360158126, 102350151728721504906083651072394940118, 125763027889175494811225472639718219003, 8449000999901250170100905516291938680, 304460600958447471089810499318960420815, 196942739825346809486608708261684069956, 95137882338323379122967612037595650372, 249103376196622054759485165738696594830, 75815255810432407877624676790418639163, 102559510410451829215697208578637690150, 79889895457239821383136285716174771255, 85759921460507891932164303125233154351, 66931880340431649952276297475508393464, 179195597392019690899901213587887843398, 42135640524923484027481717478848698135, 180762862914817191552531138388597130902, 32960818887870364452472451824158350194, 65491261711391983269874220734766808764, 75057724181153399982117676529106863901, 249203432226765999357683552550389362355, 188784211432672968015683615600804669640, 44868363512226712081688662066408303740, 312542681188936062841981754509496038915, 214468130979694090127495476216322084704, 147130926574593262530511496519200686150, 420628557000937785927222189298161930, 257314056214649715552674477675253804327, 130126886588978663121253180423219491395, 188756873783909007971080819555596875991, 77340699181935569386206419117214063920, 38405539368077037303395928229417638465, 116686145369002310970889434084130448674, 323914460335992008802272173800700035519, 201068458470759857865872089463118832542, 151541853255881851324594665697796942945, 102382733750720561016669275035278008742, 86553774561732861911725873249743334535, 37408164157533252701794004769802665127, 82990862024469281135937998499609151127, 19771487691041485820610627311382150245, 151660103043953227281983572405642625222, 257653791471817629400008361121326777985, 129725592634518800564203530464946102982, 40150321655333663919157774615420055014, 191200228421984736795897903701450505357, 295456314396482330319701792605840570814, 297793849452745763336982943164805511706, 239275084947363340329764281850641485978, 59960751737468025959771253181665979723, 268183119779877737618270397879304452863, 66224307378015102806009890220170769047, 8473563335414588479583606884249303418, 257542474584531565342158365140348246768, 105417186583593522327263886624253879933, 73032353578004845893212482954223685473, 127029393301668416574316074925305703768, 321340685168541028569915558863938397450, 167877817579307881824388394789134152926, 331413026134391251447004034759820969079, 144738178339996144920370939339285322576, 53276719569647270150548188272108203070, 233796437522410673289154776757770251104, 184283260216606000033930530973479433556, 7357949516640266756658412231053128667, 198844627404279745841847001764682713832, 71056502381608354828971162467357346951, 8582778544219429085414179091392805761, 250106113334122022488912139880016629998, 310502395565898884824487545367613096173, 187256115781292065968366673172185241106, 52272613027167213475522146255435512427, 106270234538347768548548936674345401043, 22558021798850396183076759921062000348, 102958995762944068379358342953940019960, 278472243532811378811001511652330967395, 314987058327101210808565443661742754666, 79049790179642710014438044649558282678, 38041929952150126346916423460545132129, 206244916981157947624840126542204193160, 309449076124536525327964387224583632163, 25118857751857904867610189310141870811, 58564057118954560868461076329109631677, 54491114616931563783261401724211759445, 202519463948533512879122997070761884666, 223972217028943320453619009768106286794, 58451365277028781088153927982695157345, 83600022673906045998547569214480401646, 258838563517511615486618038679333776261, 123296056973322290332407730368097290113, 70146468844777234125323106726296270216, 122331050208088502275720505166233715912, 254972000965704960014705509729685921773, 258566027411666370228708228165337679877, 128610571170515600253231267229104272452, 152863529815390653114832224530292063493, 248513969185447786784514351271906941230, 90183811177201036406221747294245395376, 138783664707918868566596812608603330132, 102306241236003058508299027565126755062, 208901907610771077740323853209080628403, 117802854715113731497309916533593659423, 64926420045175761929274218549040767994, 214390284112639524652716952553602480220, 98296922587754921816886183270712874847, 12816326430961689822298311200695985132, 70363052377696044325840271660391558537, 161697855134533916263114061685164198456, 181395834004762861169990880456319818449, 102628696760343893579901326839433152083, 68969843779248747746227575957794150090, 252138493647137508472532100794247427314, 36150326711052239038054147057005464521, 16509232194973852943074700709901869018, 190007055144778772598824161657548368727, 99385764191515426631745573758658848442, 64776265941298121517858921185842352301, 29016418264760374643750726343405408213, 295378775021822481479779264561253236114, 142074129255469384352212891137130613836, 124683216585181388060892692481419335728, 174292152129431919582081739233162387409, 171106010419072464876447734458284951220, 327776499445407370813928665450557707597, 118612709335866045996290651277735059044, 131431574119353445508307076105561052382, 160193826976095274444347092238447410666, 152340512750923007150817914357059487724, 336697964941082448363857429107050955063, 68082302409134359255592470726939740945, 331647429588254656478346698294788919769, 223643920111155584956603199167763891180, 148821434814461496304576923437674859517, 21364153629041003152791456780390815107, 76469757167238691283071266642666724561, 51429316287867532562544015407946684677, 237941849507607487088144796639886000567, 112853080877947888197817166407127104133, 204995593192843354462807992408073216073, 243329101296670762694529267098675697666, 281939084586060399666544596436950558642, 51009252244809384256410919576309560722, 334288938660597899900744376248325224688, 215198318158466698628858411931296473879, 310205805679433856927743212974604533990, 329041015016162172082311851390578426411, 210737490933928677033493421002458738204, 217106744528749745958163334465181685037, 228182742335196452039013002581528351805, 30309899318118666824524928872187328891, 100851200460251232257250883878077851577, 29379962499076378090224543445345654051, 115649760931292624714357922373980985309, 104232446854837023321813297913833520026, 250304514619257971484046831661002823971, 99494761276931490185752525210988702086, 218748577286754012906876257620231475759, 270392969550281773939979667629093535744, 114506260484840969987002634181453759866, 133523284656849404052479343371156872471, 192606723248345484968922798388911067335, 23152767873964334628606169942623094902, 227418376716927962798091333509769337005, 257135299148705581247118728231378131208, 30616546601050008188533151036257347341, 81334031322754081150316160915314429336, 205905393438210666533896067119689761335, 283034090268152004472390301883775350477, 114456167418989567844672378202171432419, 232755271332479341638243445972093014859, 268945821918166284601601185313267009612, 193739447564013619122902163012269562481, 331366033681885969490820497495293309472, 74439644705715272501685015067612606929, 50053476828869628874451394746911116174, 271771495569953959790375546169454897931, 155033735548162904441081777634162275490, 2135106266389281659555307842364320691, 238328949678813701828456159346822606571, 304522316279473989030053760522859162576, 65403109876227102396456173207582325048, 43568006146497883307180193196928278049, 121386283896773138397469620736604973961, 175579328117083688241341162132890894113, 280696411055419774386619691431207970914, 197257234468361292311668120210953783352, 254651881466686627765974137622794842307, 69786471989837051406890968843656387119, 143720180279522922542888457523856425361, 136700574323233779392141687922564891273, 72234912235859479720846970170342280895, 287406972122108852245371774452984516503, 155991133397502121792571095777610118815, 17516380298987626037693036839434643987, 30059804614330282058052362700881215461, 316486135306338288488912428312693094179, 163509469675871267839507171932364382696, 97430650571213596158972300737133158371, 63744106842238153609635888500267345842, 111721469592513399926885945842797047522, 196046285369577914623425105379446007871, 33288719344503098554509983072439722753, 214280043882766488360559334638346673217, 237523818919254504200731714101366497207, 144382159303174748311029204427113498037, 180150810200556833126894957140038127251, 328550724382623356003225068969076300215, 280182120166028642734697757623726919372, 227557580793456904979525374974603684980, 161697802423254657454611286696131533955, 20300031567207822999908882938149949195, 327583857494389997971218415875392396283, 318212243297574477388501627146362415600, 32072361868511944664444405400023557601, 187493930882403239679885046064841363127, 336683046533015455440145026319758831489, 282438850251083286220210776528563255770, 135682803472279194934993278830642105059, 104195182931093669029550541157934322728, 37822097517856769445356157035333789461, 242514864292260977252433284338835832531, 8136982945703266635174947841497334658, 124611707772541482780597321700094500543, 4769102800773783987981739105144214141, 26440156826292233002977162312519412397, 231966807454677246002908637943455860054, 54992641861510213981072324654567330145, 231359298624589996980570203025662833388, 8040220789511744953740213878519238111, 186413308242825511254329416228133181613, 5516639902170711997336778432913459732, 6892670275806708792621333501673940617, 157826151287978958172971486750596553893, 171226598956946946152435430948730566815, 115199461227557750428156206275411825245, 50606047774381098165633262252470768246, 271822731118193172360121318264435905260, 24459387336694079395216056393590644576, 329609352184270398532199555908505335504, 229797923148311498780657407194694313949, 186135252647226866194907776106507798521, 186517380182789338240302272002180364760, 10400263993343770347989678234717981944, 57674021302834516920970833893042912303, 129866496046550115011029303156159549411, 115312206883890619339042024147058576464, 253109910367331717360010393545383961610, 5280327160293598987255462331962344826, 102014667666272670241183139223528250946, 166114973377064186949589266504983568859, 22713728541861875183936181498327375084, 236440836749164074303802146915158855465, 60339130843974654382985709571166577636, 42878314096295586009646176149871422844, 306434554642634647405652078180909057478, 281170454377640413428681913194930119801, 19947969170865162290026564487295902448, 302341656265931790216895049735574817580, 247057639408508513107535475866236576540, 175548326088957264498155829888093180837, 49465692107043254948276115406645480590, 239691574493403815381378175468211648316, 228076014371349266109235716609583720474, 181736917048944488966085756377165443006, 199125994823545200988197212243063613236, 325345910633631606169309184594770884416, 227601812530702361433679178559271749525, 208573116971824405042254617300614311476, 229666813477048601106562976852606713893, 57457454131454592334880462145759269740, 22385477040747857060211715382909006798, 337126984410316496596313848641331957675, 328410293380141084614394746257333222810, 24610306489716184142480574222949237499, 176992454161329983875175178220952222654, 161553626109110697180881560191917136365, 325230740495953430991114671884677314978, 265757398540170862984730719662712436497, 109950647429789929189823839243913661212, 71049161126275388468578684863979197116, 323574947539717666401574508773296409771, 89890604014611642664773646845705156268, 27635160872165473349410368035018250914, 17196396944688361806893183264384546644, 82791886289651049108052978522392053322, 113112092734694021949395346730601254907, 275365350930046124922061384776503265451, 340022497067118422440726744608607415064, 268121172085808955578282854212062678958, 167178355255739424536487586060893985685, 234705665678054981224455499967982879033, 121611357113002239561172002710231911570, 204912114706953600514376112144931502323, 283299374609470200634219166477795620599, 53892384581585732101362072075306054049, 200782085727769333250369115068717366598, 3208087054528787090204544522150210036, 21622409785244113246164558615561073379, 48592779398876318372282435115565985263, 160372564692376834686198887197306989857, 198489833894012313393303678911078787815, 77750356798780107739150478755917955944, 134194967867907344045941976352424483499, 77307843600219696664880063317778200783, 288560300386865041757082092738247499471, 12926775156437060399728542530867485314, 80449144927817724518360683985105730537, 169970753434108921410598939044704844512, 158446482238473274363332525756190214780, 137713034185966166337701802419850865147, 288636927644943893213856733619886489278, 33703773420621532664915951257141577121, 335466635033866483761758041565265515173, 241444736604079489672556576932246672331, 292062003279448307282988593861675813770, 243516449657420869820441807676876987175, 307852754623194915309942695599924313140, 300412793158795987133383212423982410103, 10659841621602501559465869373081256407, 181239370228831107499292303102612941671, 193569611192968106562366168822669950330, 231533337969132596361303328051271372600, 256842487401277919621244327507321412480, 86934343980447294502324214031696302228, 277493194176211151254243964122603056624, 48993270312153464417685703080560161551, 112977343517350548761356562685798233821, 282304060674858554572173065555650124273, 100523389372189404112212846386354975533, 199706669254875897640295693181720898388, 318910543633336907590696797988533649133, 113740230057651002184762171876537941053, 188332501509438171300648393495896639342, 63158702492156917640045632675009500838, 274925457261174948146138258227455436296, 261821189613178829532905884606849509606, 242166691585217269942961801485060241984, 135195446406356621400062470737344716437, 104738727048375014711715007033756054322, 291152250615463168970280853317903207757, 22811157396694605801609978640826703178, 216151867224220008885373356203866686632, 11276654699404524569956949004465681822, 293758660633650872126016544835090868013, 137768055994430884294804948203858493226, 70115238092063690226164246795051950458, 235189015413519649437153343462627438777, 39197912237965751631648843581443273915, 85369006881254895435897173223659203959, 178227648578050647405141964750112567365, 160498532792052956363842473852479080357, 102762075905897429127866689445736959046, 291097712558476267210692038643274009673, 233149058808315648350557992111553104569, 268067551638129604135201595494002161927, 64426162659397100162510696273475569056, 31029189438073315852000083021527430349, 248839412228027669909491537967894302023, 191566018949813570801452786659787283194, 186665095866829840315253403502513126283, 87067810426970024052000516587308696918, 223142581220694916321246961527951922498, 157841058826550144468236917003227939375, 26508186584292600707506928657762074792, 80623607170133611156299492068311688297, 314959176123103473273116929723438648608, 204409502504563095497664813802718042629, 225022022668096393133914078806082679098, 127920694649915281027085943723177901853, 199901720064877101190258854823861381960, 240066429711491005781834150183637221771, 111102194425245488677064116927441798827, 67119865551706707876102263088644795683, 211226343780290880626050133625857242, 147462210951530432510640079146512763678, 223239247435584046267089396985906049921, 31528008042551405770985007372187243148, 17765905295172199576048912208265073104, 163573766888453487015454388560679316139, 162199325530665262262847588773246840765, 254740319713803491649106888689415908954, 172982582852922355494770778937102502353, 72274748910227911476297334015556105277, 230994513915880211411056493925453841576, 125523970907075743674059144307983437585, 204545130366956585116021430650129949657, 281769384067015634990551866449570497040, 79091019582532942693301724850924443210, 2451818452559085758158214760471687307, 295234675854427222396981580828780591405, 330509237644910969486916023372995113426, 118170477795842557560664350683197636016, 285156858544549585195566657976154994658, 65369695065097831659792098126946023966, 166234621022507035464560026946880353057, 126147485160930179923036213074029463463, 284196375017370613300419168602010731302, 251370557139805799342811610015223234603, 125025121289253151207295450101973019322, 45335255091332744252399973823111076529, 331978747976202611098636515535594207982, 285913993793576824416481594829544103296, 161926544625613481237832698780352613901, 197933694710308674019477637564286175424, 170011699232673290973248550476151287829, 80420821702048900373518285379279118998, 204425784984406283361641707757952322276, 113502189090424657793755667932611168072, 331723969744029674368003689254418199321, 12553263091228807296990132582055094330, 263999030506700511592043209623144282402, 89663766256726191415905297624876647427, 198693743930333852037566815953003726771, 35376403794336669140301220448867117439, 292272890902819891769348397728946304060, 263601311568355651348653578366375772808, 167309340207566457404702917541068802376, 116946864392641412596484373531728829706, 239486201591249998862484345161180422397, 200726556038852785925820231463248020756, 290763194174213134417709052188979863894, 10980897775904828922204226674614933367, 320229022952368703576093758663216434017, 233202755387982930080886647255166327322, 269975398753808036173395902719900095846, 33269697005321706858439887286025979877, 214163878539483463891231576676204561694, 273835311928994016294282677498191864330, 142156793534777840817108927432471676315, 226283562951760194092694628680463980293, 63288576733844468578917135034816591674, 215703645202286896332192821437756893735, 86480306794744234449643136449811272827, 53176085577733408051337920114021503938, 317064761394586275211376850710160547289, 47297569197416492456317254425116237838, 279493982196569678066834474851235396336, 105589887776470428218272964846247317968, 312974293644018395128054531777440730501, 231503368339489519343267780746215870672, 246491194035767324813769731766990132627, 249484555901000617678045650899231849991, 60761955800208843849669842070355177412, 294313043895338686938541605740338303340, 249996323199143052278590833341969803238, 254263526928856575007492226361204881712, 253308214626258196769889578133866125489, 163794866083069953729679567300426672561, 70836290150428865882991704663791114278, 224590859236935467015102479404770572115, 76368114078778482951909523413244744527, 50054208368908103788601317518821412870, 110417906106249136984689720023801665117, 238852080488994544077375120285355275895, 122610888301904271936270583208259504483, 328830414450263177730561100470115345899, 79479914936902066070256231280763249823, 304466148380426460837273237585204595469, 337615869084715031130843299498004085633, 109008019560800689654722541271717899965, 302547519421100431278683730894670553110, 131732281260238423038259267998391364377, 30121033180040203737739894792566035348, 56572221143820879441248636208788254150, 240640636222260153578035661062643018927, 225760932802168750524402654476848054135, 54424483887022515734764558810048846647, 226239273817608687657156053625640075008, 78201303895194178962496865372720036093, 22648206523922646964407362180258677700, 128003378853944265094986747276415261851, 287447248769205635812284205547929938347, 47626455939656498335803636859212797500, 156387131635280908756744780695048177727, 67197479489813155809711993338991677374, 95300294073641119315556220419934693093, 64966940804075589464381714501649779960, 149703407072444506002073480012244436332, 152043030777976315212362984341570965482, 83876710179566193706674534443163716032, 222897921640759792080172540523866961454, 175956785210825497968830066992305025177, 100825439417467756676023533257147984641, 236012744484282016979294887044250736771, 257914421533615803979178988955488132575, 70988706041790567199621435921130468530, 133499347489600615085764599528289428033, 249345167688246658216529675848211461974, 15165000905920554953334102523572269701, 186137817758944100187313689045316270486, 305848532757626134471444850355167566389, 146619263457851991808695270170811357848, 258186371814849006430607486554084869561, 83315566443416231819561741536646853109, 112715626738203440324106391056132762549, 204420861604892106365508869271457802627, 46849649893977991536009233570556756631, 306408593759400532865275414887343441344, 75069329177871367055782059702038383734, 8968465957378882057544190393864894501, 240223711781160397153531417318555139387, 322591101158675611744666137261239268369, 201126503043759227356293450510213927365, 277893853046125602206941410810192113967, 295000329019440157647598663354984252566, 817616170561677417740370995054272030, 261475489004255070957722624604499436362, 262567259551943377693122640068350186792, 153250655087314029344952347171594708847, 134295881441428978659484144885147328332, 298883695794923509444664963870250810513, 48033632774258312129962074809227426033, 167602431751251688711811714987000491069, 57441148671560759961034119334042324911, 50815687127512052931250638249757058374, 106703997612358636382906710129822997942, 849790230275433188117496864044956870, 246482297269381625076756133036170121760, 84830594078563246978311928523618144945, 187399384777967577930385285353843349673, 30718935185045464550905724210261762945, 236109873344381775735911821442869562230, 123572993455706491602244749812255123382, 140774048915267390170023336945516530111, 16929879941418954431949554933590983196, 11190370258001868756798023474534428604, 33105424368811981468773166225632681302, 179883347268229044144685893584668427441, 270769311161852384622723753387786180952, 97677958755783053527362436452048594360, 3823572627051584013220337228663191364, 155477196814069732261220824755825456760, 19478809682760423335159092683860445005, 88585164532895812192091007054022029558, 179163558081352395450749560780634203935, 187157361340619950035052112219420969621, 168930978822027590835827747669882821778, 103656660997933743393271919255412997683, 25300937227642267253011820447665830009, 139811117970714275435653905383447305381, 43905484863972861620809647466283668162, 14551580529524478011925581921303222120, 81127303999735736981815891632798282231, 236736382518793948199578012964975394111, 269405690545770695298109681009047906525, 175826574332539982480549759083865843863, 55990369453160421322748278848687971452, 30424943819295584190372198603636286309, 203908154919727759564845732817038292525, 263905678887953318655437096402287346803, 16388784073388736082205664241284479480, 303886224585167423587457338807084853834, 217959521344625872494801143882704239077, 307643851995773752914366264810043738041, 205878623239006925702372941805457638968, 163668194054922112521673610813701236737, 338183532460381362967502699097111816137, 293942596474209921939569306098107525479, 256779216142103032022375410705079289644, 128965296785077944306388900345602229590, 101442366310523545012580941926518355062, 61497080799953016611196008491219747483, 205544351246993653600253753248542283075, 1515962108092291849256045979182145327, 146552970226719642333060767064259085670, 186718402305634920337667456483932666464, 44775973760634110712572092506518786570, 154572432427404213435653223843323525044, 286629507993705407164402732180494826725, 136397683498033556765977897476771366809, 255389712349106810760630016947981293245], [4037797258022100113842494415863664385, 288027277195630534052428568134696149071, 192066837354053545233955008532086692251], [243382710732531489693195870365903964183, 132302907946296948163998721551176913917], [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 88, 92], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [1, 2, 3, 4], [0, 1], [0, 1]], missing_values_reference_list=['', '-', '?', nan])))
    preprocessor_features_categorical_steps.append(('boolean2float_transformer', autoai_libs.transformers.exportable.boolean2float(activate_flag=True)))
    preprocessor_features_categorical_steps.append(('cat_imputer', autoai_libs.transformers.exportable.CatImputer(activate_flag=True, missing_values=nan, sklearn_version_family='23', strategy='most_frequent')))
    preprocessor_features_categorical_steps.append(('cat_encoder', autoai_libs.transformers.exportable.CatEncoder(activate_flag=True, categories='auto', dtype=numpy.float64, encoding='ordinal', handle_unknown='error', sklearn_version_family='23')))
    preprocessor_features_categorical_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_categorical_ Pipeline
    preprocessor_features_categorical_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_categorical_steps)
    preprocessor_features_transformer_list.append(('categorical', preprocessor_features_categorical_pipeline))
    #
    # composing steps for preprocessor_features_numeric Pipeline
    #
    preprocessor_features_numeric__input_metadata = None
    preprocessor_features_numeric_steps = []
    preprocessor_features_numeric_steps.append(('num_column_selector', autoai_libs.transformers.exportable.NumpyColumnSelector(columns=[0, 2, 7, 11])))
    preprocessor_features_numeric_steps.append(('num_floatstr2float_transformer', autoai_libs.transformers.exportable.FloatStr2Float(activate_flag=True, dtypes_list=['int_num', 'int_num', 'float_num', 'float_num'], missing_values_reference_list=[])))
    preprocessor_features_numeric_steps.append(('num_missing_replacer', autoai_libs.transformers.exportable.NumpyReplaceMissingValues(filling_values=nan, missing_values=[])))
    preprocessor_features_numeric_steps.append(('num_imputer', autoai_libs.transformers.exportable.NumImputer(activate_flag=True, missing_values=nan, strategy='median')))
    preprocessor_features_numeric_steps.append(('num_scaler', autoai_libs.transformers.exportable.OptStandardScaler(num_scaler_copy=None, num_scaler_with_mean=None, num_scaler_with_std=None, use_scaler_flag=False)))
    preprocessor_features_numeric_steps.append(('float32_transformer', autoai_libs.transformers.exportable.float32_transform(activate_flag=True)))
    # assembling preprocessor_features_numeric_ Pipeline
    preprocessor_features_numeric_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_features_numeric_steps)
    preprocessor_features_transformer_list.append(('numeric', preprocessor_features_numeric_pipeline))
    # assembling preprocessor_features_ FeatureUnion
    preprocessor_features_pipeline = sklearn.pipeline.FeatureUnion(transformer_list=preprocessor_features_transformer_list)
    preprocessor_steps.append(('features', preprocessor_features_pipeline))
    preprocessor_steps.append(('permuter', autoai_libs.transformers.exportable.NumpyPermuteArray(axis=0, permutation_indices=[1, 3, 4, 5, 6, 8, 9, 10, 0, 2, 7, 11])))
    # assembling preprocessor_ Pipeline
    preprocessor_pipeline = sklearn.pipeline.Pipeline(steps=preprocessor_steps)
    steps.append(('preprocessor', preprocessor_pipeline))
    #
    # composing steps for cognito Pipeline
    #
    cognito__input_metadata = None
    cognito_steps = []
    cognito_steps.append(('0', autoai_libs.cognito.transforms.transform_utils.TA1(fun=numpy.tan, name='tan', datatypes=['float'], feat_constraints=[autoai_libs.utils.fc_methods.is_not_categorical], tgraph=None, apply_all=True, col_names=['CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'], col_dtypes=[dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32'), dtype('float32')], col_as_json_objects=None)))
    cognito_steps.append(('1', autoai_libs.cognito.transforms.transform_utils.FS1(cols_ids_must_keep=range(0, 12), additional_col_count_to_keep=12, ptype='classification')))
    # assembling cognito_ Pipeline
    cognito_pipeline = sklearn.pipeline.Pipeline(steps=cognito_steps)
    steps.append(('cognito', cognito_pipeline))
    steps.append(('estimator', sklearn.ensemble._gb.GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None, learning_rate=0.1, loss='exponential', max_depth=3, max_features=0.9999999989834419, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=0.010000000267767636, min_samples_split=0.010000000142768466, min_weight_fraction_leaf=0.0, n_estimators=80, n_iter_no_change=None, presort='auto', random_state=33, subsample=1.0, tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False)))
    # assembling  Pipeline
    pipeline = sklearn.pipeline.Pipeline(steps=steps)
    return pipeline
pipeline = compose_pipeline()


### 3. Extract needed parameter values from AutoAI run metadata

In [ ]:

# Metadata used in retrieving data and computing metrics.  Customize as necessary for your environment.
#data_source='replace_with_path_and_csv_filename'
target_label_name = _input_metadata['target_label_name']
learning_type = _input_metadata['learning_type']
optimization_metric = _input_metadata['optimization_metric']
random_state = _input_metadata['random_state']
cv_num_folds = _input_metadata['cv_num_folds']
holdout_fraction = _input_metadata['holdout_fraction']
if 'data_provenance' in _input_metadata:
    data_provenance = _input_metadata['data_provenance']
else:
    data_provenance = None
if 'pos_label' in _input_metadata and learning_type == 'classification':
    pos_label = _input_metadata['pos_label']
else:
    pos_label = None


### 4. Create dataframe from dataset in Cloud Object Storage

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_0 = {
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.softlayer.net',
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token/',
    'APIKEY': 'C3zLD83pMr7UVInk1mYu6Q_gPA9rvR9ZqoxcPzG9u5vf',
    'BUCKET': 'telecomcustomerchurnprediction-donotdelete-pr-mbvsomwr0akqre',
    'FILE': 'Churn_Modelling.csv',
    'SERVICE_NAME': 's3',
    'ASSET_ID': '1',
    }


In [ ]:
#  Read the data as a dataframe
import pandas as pd

csv_encodings=['UTF-8','Latin-1'] # supplement list of encodings as necessary for your data
df = None
readable = None  # if automatic detection fails, you can supply a filename here

# First, obtain a readable object
# Cloud Object Storage data access
# Assumes COS credentials are in a dictionary named 'credentials_0'
   
credentials = df = globals().get('credentials_0')       
if readable is None and credentials is not None :
    try:
        import types
        import pandas as pd
        import io
    except Exception as import_exception:
        print('Error with importing packages - check if you installed them on your environment')
    try:
        if credentials['SERVICE_NAME'] == 's3':
            try:
                from botocore.client import Config
                import ibm_boto3
            except Exception as import_exception:
                print('Installing required packages!')
                !pip install ibm-cos-sdk
                print('accessing data via Cloud Object Storage')
            try:
                client = ibm_boto3.client(service_name=credentials['SERVICE_NAME'],
                                    ibm_api_key_id=credentials['APIKEY'],
                                    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
                                    config=Config(signature_version='oauth'),
                                    endpoint_url=credentials['ENDPOINT'])
            except Exception as cos_exception:
                print('unable to create client for cloud object storage')
            try:
                readable = client.get_object(Bucket=credentials['BUCKET'],Key=credentials['FILE'])['Body']
                # add missing __iter__ method, so pandas accepts readable as file-like object
                if not hasattr(readable, "__iter__"): readable.__iter__ = types.MethodType( __iter__, readable )
            except Exception as cos_access_exception:
                print('unable to access data object in cloud object storage with credentials supplied')
        elif credentials['SERVICE_NAME'] == 'fs':
            print('accessing data via File System')
            try:
                if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                    df = pd.read_excel(credentials['FILE'])
                else:
                    df = pd.read_csv(credentials['FILE'], sep = _input_metadata['separator'])
            except Exception as FS_access_exception:
                print('unable to access data object in File System with path supplied') 
    except Exception as data_access_exception:
        print('unable to access data object with credentials supplied') 

# IBM Cloud Pak for Data data access
project_filename = globals().get('project_filename')       
if readable is None and 'credentials_0' in globals() and 'ASSET_ID' in credentials_0:
    project_filename = credentials_0['ASSET_ID']
if project_filename != None and project_filename != '1':
    print('attempting project_lib access to ' + str(project_filename))
    try:
        from project_lib import Project
        project = Project.access()
        storage_credentials = project.get_storage_metadata()
        readable = project.get_file(project_filename)
    except Exception as project_exception:
        print('unable to access data using the project_lib interface and filename supplied')

# Use data_provenance as filename if other access mechanisms are unsuccessful
if readable is None and type(data_provenance) is str:
    print('attempting to access local file using path and name ' + data_provenance)
    readable = data_provenance

# Second, use pd.read_csv to read object, iterating over list of csv_encodings until successful
if readable is not None:
    for encoding in csv_encodings:
        try:
            if credentials['FILE'].endswith('xlsx') or credentials['FILE'].endswith('xls'):
                buffer = io.BytesIO(readable.read())
                buffer.seek(0)
                df = pd.read_excel(buffer, encoding=encoding,sheet_name=_input_metadata['excel_sheet'])
            else:
                df = pd.read_csv(readable, encoding = encoding, sep = _input_metadata['separator'])
            print('successfully loaded dataframe using encoding = ' + str(encoding))
            break
        except Exception as exception_dataread:
            print('unable to read csv using encoding ' + str(encoding))
            print('handled error was ' + str(exception_dataread))
    if df is None:
        print('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings))
        print(f'Please use \'insert to code\' on data panel to load dataframe.')
        raise(ValueError('unable to read file/object as a dataframe using supplied csv_encodings ' + str(csv_encodings)))

if isinstance(df,pd.DataFrame):
    print('Data loaded succesfully')
    if _input_metadata.get('subsampling') is not None:
        df = df.sample(frac=_input_metadata['subsampling'], random_state=_input_metadata['random_state']) if _input_metadata['subsampling'] <= 1.0 else df.sample(n=_input_metadata['subsampling'], random_state=_input_metadata['random_state'])
else:
    print('Data cannot be loaded with credentials supplied, please provide DataFrame with training data.')

### 5. Preprocess Data

In [ ]:
# Drop rows whose target is not defined
target = target_label_name # your target name here
if learning_type == 'regression':
    df[target] = pd.to_numeric(df[target], errors='coerce')
df.dropna('rows', how='any', subset=[target], inplace=True)


In [ ]:
# extract X and y
df_X = df.drop(columns=[target])
df_y = df[target]


In [ ]:
# Detach preprocessing pipeline (which needs to see all training data)
preprocessor_index = -1
preprocessing_steps = [] 
for i, step in enumerate(pipeline.steps):
    preprocessing_steps.append(step)
    if step[0]=='preprocessor':
        preprocessor_index = i
        break
#if len(pipeline.steps) > preprocessor_index+1 and pipeline.steps[preprocessor_index + 1][0] == 'cognito':
    #preprocessor_index += 1
    #preprocessing_steps.append(pipeline.steps[preprocessor_index])
if preprocessor_index >= 0:
    preprocessing_pipeline = Pipeline(memory=pipeline.memory, steps=preprocessing_steps)
    pipeline = Pipeline(steps=pipeline.steps[preprocessor_index+1:])

In [ ]:
# Preprocess X
# preprocessor should see all data for cross_validate on the remaining steps to match autoai scores
known_values_list.clear()  #  known_values_list is filled in by the preprocessing_pipeline if needed
preprocessing_pipeline.fit(df_X.values, df_y.values)
X_prep = preprocessing_pipeline.transform(df_X.values)

### 6. Split data into Training and Holdout sets

In [ ]:
# determine learning_type and perform holdout split (stratify conditionally)
if learning_type is None:
    # When the problem type is not available in the metadata, use the sklearn type_of_target to determine whether to stratify the holdout split
    # Caution:  This can mis-classify regression targets that can be expressed as integers as multiclass, in which case manually override the learning_type
    from sklearn.utils.multiclass import type_of_target
    if type_of_target(df_y.values) in ['multiclass', 'binary']:
        learning_type = 'classification'
    else:
        learning_type = 'regression'
    print('learning_type determined by type_of_target as:',learning_type)
else:
    print('learning_type specified as:',learning_type)
    
from sklearn.model_selection import train_test_split
if learning_type == 'classification':
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state, stratify=df_y.values)
else:
    X, X_holdout, y, y_holdout = train_test_split(X_prep, df_y.values, test_size=holdout_fraction, random_state=random_state)


#### 7. Generate features via Feature Engineering pipeline

In [ ]:
#Detach Feature Engineering pipeline if next, fit it, and transform the training data
fe_pipeline = None
if pipeline.steps[0][0] == 'cognito':
    try:
        fe_pipeline = Pipeline(steps=[pipeline.steps[0]])
        X = fe_pipeline.fit_transform(X, y)
        X_holdout = fe_pipeline.transform(X_holdout)
        pipeline.steps = pipeline.steps[1:]
    except IndexError:
        try:
            print('Trying to compose pipeline with some of cognito steps')
            fe_pipeline = Pipeline(steps = list([pipeline.steps[0][1].steps[0],pipeline.steps[0][1].steps[1]]))
            X = fe_pipeline.fit_transform(X, y)
            X_holdout = fe_pipeline.transform(X_holdout)
            pipeline.steps = pipeline.steps[1:]
        except IndexError:
            print('Composing pipeline without cognito steps!')
            pipeline.steps = pipeline.steps[1:]


 ### 8. Additional setup: Define a function that returns a scorer for the target's positive label

In [ ]:
# create a function to produce a scorer for a given positive label
def make_pos_label_scorer(scorer, pos_label):
    kwargs = {'pos_label':pos_label}
    for prop in ['needs_proba', 'needs_threshold']:
        if prop+'=True' in scorer._factory_args():
            kwargs[prop] = True
    if scorer._sign == -1:
        kwargs['greater_is_better'] = False
    from sklearn.metrics import make_scorer
    scorer=make_scorer(scorer._score_func, **kwargs)
    return scorer

### 9. Fit pipeline, predict on Holdout set, calculate score, perform cross-validation

In [ ]:
# fit the remainder of the pipeline on the training data
pipeline.fit(X,y)

In [ ]:
# predict on the holdout data
y_pred = pipeline.predict(X_holdout)

In [ ]:
# compute score for the optimization metric
# scorer may need pos_label, but not all scorers take pos_label parameter
from sklearn.metrics import get_scorer
scorer = get_scorer(optimization_metric)
score = None
#score = scorer(pipeline, X_holdout, y_holdout)  # this would suffice for simple cases
pos_label = None  # if you want to supply the pos_label, specify it here
if pos_label is None and 'pos_label' in _input_metadata:
    pos_label=_input_metadata['pos_label']
try:
    score = scorer(pipeline, X_holdout, y_holdout)
except Exception as e1:
    if learning_type is "classification" and (pos_label is None or str(pos_label)==''):
        print('You may have to provide a value for pos_label in order for a score to be calculated.')
        raise(e1)
    else:
        exception_string=str(e1)
        if 'pos_label' in exception_string:
            try:
                scorer = make_pos_label_scorer(scorer, pos_label=pos_label)
                score = scorer(pipeline, X_holdout, y_holdout)
                print('Retry was successful with pos_label supplied to scorer')
            except Exception as e2:
                print('Initial attempt to use scorer failed.  Exception was:')
                print(e1)
                print('')
                print('Retry with pos_label failed.  Exception was:')
                print(e2)
        else:
            raise(e1)

if score is not None:
    print(score)

In [ ]:
# cross_validate pipeline using training data
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold, KFold
if learning_type == 'classification':
    fold_generator = StratifiedKFold(n_splits=cv_num_folds, random_state=random_state)
else:
    fold_generator = KFold(n_splits=cv_num_folds, random_state=random_state)
cv_results = cross_validate(pipeline, X, y, cv=fold_generator, scoring={optimization_metric:scorer}, return_train_score=True)
import numpy as np
np.mean(cv_results['test_' + optimization_metric])

In [ ]:
cv_results